# 1. DBBE

In [ ]:
from sklearn.metrics import adjusted_rand_score, v_measure_score
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
from typing import List, Dict, Tuple, Optional, Set
import time
from tqdm import tqdm
import unicodedata
from pathlib import Path

RESULTS_DIR = Path("dbbe_orthographic_results")
RESULTS_DIR.mkdir(exist_ok=True)

try:
    import cupy as cp
    GPU_AVAILABLE = True
    print("GPU detected - using CuPy acceleration")
except ImportError:
    cp = np
    GPU_AVAILABLE = False
    print("No GPU - using NumPy (CPU mode)")


class TextPreprocessor:
    def __init__(self, lowercase=True, remove_punctuation=True, remove_diacritics=True):
        self.lowercase = lowercase
        self.remove_punctuation = remove_punctuation
        self.remove_diacritics = remove_diacritics
        if remove_punctuation:
            self.punct_pattern = re.compile(r'[^\w\s]', re.UNICODE)
            self.remove_chars_pattern = re.compile(r'[\(\)\{\}]')

    def _remove_diacritics(self, text: str) -> str:
        return ''.join(
            c for c in unicodedata.normalize('NFD', text)
            if unicodedata.category(c) != 'Mn'
        )

    def preprocess(self, text: str) -> str:
        if not isinstance(text, str):
            text = str(text) if pd.notna(text) else ''

        if self.remove_diacritics:
            text = self._remove_diacritics(text)
        if self.lowercase:
            text = text.lower()
        if self.remove_punctuation:
            text = self.remove_chars_pattern.sub('', text)
            text = self.punct_pattern.sub(' ', text)

        return ' '.join(text.split())

    def preprocess_batch(self, texts: List[str]) -> List[str]:
        return [self.preprocess(t) for t in texts]


class ShingleGenerator:
    def __init__(self, shingle_size: int = 4, use_gpu: bool = GPU_AVAILABLE):
        self.shingle_size = shingle_size
        self.use_gpu = use_gpu and GPU_AVAILABLE
        self.xp = cp if self.use_gpu else np

    def generate_shingles(self, text: str) -> np.ndarray:
        if len(text) < self.shingle_size:
            return np.array([hash(text) % (2**31)], dtype=np.int32)

        chars = self.xp.array([ord(c) for c in text], dtype=np.int32)
        n_shingles = len(text) - self.shingle_size + 1

        shingles = self.xp.zeros(n_shingles, dtype=np.int32)
        for i in range(self.shingle_size):
            shingles += chars[i:i+n_shingles] * (31 ** i)

        unique_shingles = self.xp.unique(shingles)

        if self.use_gpu:
            unique_shingles = cp.asnumpy(unique_shingles)

        return unique_shingles

    def generate_batch(self, texts: List[str]) -> List[np.ndarray]:
        return [self.generate_shingles(t) for t in texts]


class MinHashProcessor:
    def __init__(self, num_perm: int = 128, use_gpu: bool = GPU_AVAILABLE):
        self.num_perm = num_perm
        self.use_gpu = use_gpu and GPU_AVAILABLE
        self.xp = cp if self.use_gpu else np

        rng = self.xp.random.RandomState(42)
        self.hash_a = rng.randint(1, 2**31-1, num_perm, dtype=np.int64)
        self.hash_b = rng.randint(0, 2**31-1, num_perm, dtype=np.int64)
        self.prime = np.int64(2**31-1)

        if self.use_gpu:
            print(f"Using GPU for MinHash ({num_perm} permutations)")

    def compute_signature(self, shingles: np.ndarray) -> np.ndarray:
        if len(shingles) == 0:
            return np.full(self.num_perm, self.prime, dtype=np.int64)

        if self.use_gpu:
            shingles_gpu = self.xp.array(shingles, dtype=np.int64)
        else:
            shingles_gpu = shingles.astype(np.int64)

        shingles_expanded = shingles_gpu[:, self.xp.newaxis]
        hashes = (self.hash_a * shingles_expanded + self.hash_b) % self.prime
        signature = self.xp.min(hashes, axis=0)

        if self.use_gpu:
            signature = cp.asnumpy(signature)

        return signature

    def compute_batch(self, shingles_batch: List[np.ndarray]) -> np.ndarray:
        signatures = np.zeros((len(shingles_batch), self.num_perm), dtype=np.int64)
        for i, shingles in enumerate(shingles_batch):
            signatures[i] = self.compute_signature(shingles)
        return signatures


class LSHIndex:
    def __init__(self, threshold: float = 0.3, num_perm: int = 128):
        self.threshold = threshold
        self.num_perm = num_perm
        self.bands = 16
        self.rows = num_perm // self.bands
        self.signatures = []
        self.num_docs = 0
        self.hash_tables = [defaultdict(list) for _ in range(self.bands)]

    def _hash_band(self, band: np.ndarray) -> int:
        return int(hash(tuple(band)) % (2**31))

    def insert_batch(self, signatures: np.ndarray, start_idx: int):
        batch_size = signatures.shape[0]
        self.signatures.append(signatures)

        for band_idx in range(self.bands):
            start_row = band_idx * self.rows
            end_row = start_row + self.rows

            for doc_idx in range(batch_size):
                band = signatures[doc_idx, start_row:end_row]
                band_hash = self._hash_band(band)
                global_doc_id = start_idx + doc_idx
                self.hash_tables[band_idx][band_hash].append(global_doc_id)

        self.num_docs += batch_size

    def query_batch(self, signatures: np.ndarray, start_idx: int) -> List[set]:
        batch_size = signatures.shape[0]
        candidates = [set() for _ in range(batch_size)]

        for band_idx in range(self.bands):
            start_row = band_idx * self.rows
            end_row = start_row + self.rows

            for doc_idx in range(batch_size):
                query_doc_id = start_idx + doc_idx
                band = signatures[doc_idx, start_row:end_row]
                band_hash = self._hash_band(band)
                bucket = self.hash_tables[band_idx].get(band_hash, [])
                candidates[doc_idx].update(c for c in bucket if c < query_doc_id)

        return candidates


class SimilarityComputer:
    def __init__(self, threshold: float = 0.3, use_gpu: bool = GPU_AVAILABLE):
        self.threshold = threshold
        self.use_gpu = use_gpu and GPU_AVAILABLE
        self.xp = cp if self.use_gpu else np

    def compute_batch_similarities(self, query_sig: np.ndarray,
                                   candidate_sigs: np.ndarray) -> np.ndarray:
        if self.use_gpu:
            query_gpu = self.xp.array(query_sig)
            cands_gpu = self.xp.array(candidate_sigs)
            query_expanded = self.xp.tile(query_gpu, (len(candidate_sigs), 1))
            matches = self.xp.sum(query_expanded == cands_gpu, axis=1)
            sims = matches.astype(np.float32) / query_sig.shape[0]
            return cp.asnumpy(sims)
        else:
            query_expanded = np.tile(query_sig, (len(candidate_sigs), 1))
            matches = np.sum(query_expanded == candidate_sigs, axis=1)
            return matches.astype(np.float32) / query_sig.shape[0]


class UnionFind:
    def __init__(self, n: int):
        self.parent = list(range(n))
        self.rank = [0] * n

    def find(self, x: int) -> int:
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, x: int, y: int):
        px, py = self.find(x), self.find(y)
        if px == py:
            return
        if self.rank[px] < self.rank[py]:
            px, py = py, px
        self.parent[py] = px
        if self.rank[px] == self.rank[py]:
            self.rank[px] += 1

    def get_clusters(self) -> Dict[int, int]:
        return {i: self.find(i) for i in range(len(self.parent))}


class FastMinHashClustering:
    def __init__(self, threshold: float = 0.3, shingle_size: int = 4,
                 num_perm: int = 128, chunk_size: int = 50000,
                 use_gpu: Optional[bool] = None):

        if use_gpu is None:
            use_gpu = GPU_AVAILABLE

        self.threshold = threshold
        self.chunk_size = chunk_size
        self.use_gpu = use_gpu and GPU_AVAILABLE

        self.preprocessor = TextPreprocessor(
            lowercase=True,
            remove_punctuation=True,
            remove_diacritics=True
        )
        self.shingler = ShingleGenerator(shingle_size, use_gpu)
        self.minhash = MinHashProcessor(num_perm, use_gpu)
        self.lsh_index = LSHIndex(threshold, num_perm)
        self.similarity_computer = SimilarityComputer(threshold, use_gpu)
        self.all_similarities = []

        mode = "GPU (CuPy)" if self.use_gpu else "CPU (NumPy)"
        print(f"Initialized in {mode} mode")

    def cluster(self, texts: List[str]) -> Tuple[Dict[int, int], List[Tuple[int, int, float]]]:
        n_docs = len(texts)
        n_chunks = (n_docs + self.chunk_size - 1) // self.chunk_size

        print(f"\nClustering {n_docs:,} documents in {n_chunks} chunks")
        print(f"threshold={self.threshold}, chunk_size={self.chunk_size:,}")

        start_time = time.time()

        for chunk_idx in tqdm(range(n_chunks), desc="Processing"):
            chunk_start = chunk_idx * self.chunk_size
            chunk_end = min(chunk_start + self.chunk_size, n_docs)
            chunk_texts = texts[chunk_start:chunk_end]

            processed = self.preprocessor.preprocess_batch(chunk_texts)
            shingles = self.shingler.generate_batch(processed)
            signatures = self.minhash.compute_batch(shingles)
            self.lsh_index.insert_batch(signatures, chunk_start)

            if chunk_start > 0:
                candidates = self.lsh_index.query_batch(signatures, chunk_start)

                for doc_idx, cand_set in enumerate(candidates):
                    if not cand_set:
                        continue

                    query_doc_id = chunk_start + doc_idx
                    query_sig = signatures[doc_idx]

                    cand_list = sorted(cand_set)
                    cand_sigs = []
                    for cand_id in cand_list:
                        batch_idx = cand_id // self.chunk_size
                        local_idx = cand_id % self.chunk_size
                        if batch_idx < len(self.lsh_index.signatures):
                            cand_sigs.append(self.lsh_index.signatures[batch_idx][local_idx])

                    if cand_sigs:
                        cand_sigs = np.array(cand_sigs)
                        sims = self.similarity_computer.compute_batch_similarities(
                            query_sig, cand_sigs
                        )

                        for cand_id, sim in zip(cand_list[:len(sims)], sims):
                            if sim >= self.threshold:
                                self.all_similarities.append((cand_id, query_doc_id, float(sim)))

        elapsed = time.time() - start_time
        print(f"\nFound {len(self.all_similarities):,} similarities in {elapsed:.2f}s")
        print(f"Throughput: {n_docs/elapsed:,.0f} docs/sec")

        print("Building clusters...")
        uf = UnionFind(n_docs)
        for doc1, doc2, _ in tqdm(self.all_similarities, desc="Clustering"):
            uf.union(doc1, doc2)

        clusters = uf.get_clusters()
        n_clusters = len(set(clusters.values()))

        total_time = time.time() - start_time
        print(f"\nCreated {n_clusters:,} clusters in {total_time:.2f}s total")

        return clusters, self.all_similarities


def reconstruct_poems(df):
    poem_to_clusters = defaultdict(set)
    poem_verse_counts = defaultdict(int)

    for _, row in df.iterrows():
        poem_id = row['idoriginal_poem']
        cluster_id = row['cluster_id']
        poem_verse_counts[poem_id] += 1
        if cluster_id != -1:
            poem_to_clusters[poem_id].add(cluster_id)

    print(f"\nReconstructed {len(poem_to_clusters)} poems")
    return poem_to_clusters, poem_verse_counts


def calculate_poem_cluster_similarity(clusters_a: Set[int], clusters_b: Set[int]) -> float:
    if not clusters_a or not clusters_b:
        return 0.0
    intersection = len(clusters_a & clusters_b)
    union = len(clusters_a | clusters_b)
    return intersection / union if union > 0 else 0.0


def cluster_poems(poem_to_clusters: Dict, similarity_threshold: float = 0.60):
    poem_ids = list(poem_to_clusters.keys())
    n_poems = len(poem_ids)

    edges = []
    for i in range(n_poems):
        for j in range(i + 1, n_poems):
            poem_a = poem_ids[i]
            poem_b = poem_ids[j]
            similarity = calculate_poem_cluster_similarity(
                poem_to_clusters[poem_a],
                poem_to_clusters[poem_b]
            )
            if similarity >= similarity_threshold:
                edges.append((poem_a, poem_b, similarity))

    class PoemUnionFind:
        def __init__(self, elements):
            self.parent = {e: e for e in elements}
            self.rank = {e: 0 for e in elements}

        def find(self, x):
            if self.parent[x] != x:
                self.parent[x] = self.find(self.parent[x])
            return self.parent[x]

        def union(self, x, y):
            px, py = self.find(x), self.find(y)
            if px == py:
                return
            if self.rank[px] < self.rank[py]:
                px, py = py, px
            self.parent[py] = px
            if self.rank[px] == self.rank[py]:
                self.rank[px] += 1

    uf = PoemUnionFind(poem_ids)
    for poem_a, poem_b, _ in edges:
        uf.union(poem_a, poem_b)

    poem_clusters = {poem_id: uf.find(poem_id) for poem_id in poem_ids}
    n_clusters = len(set(poem_clusters.values()))

    return poem_clusters, edges, n_clusters


def evaluate_clustering(y_true, y_pred):
    ari = adjusted_rand_score(y_true, y_pred)
    v_measure = v_measure_score(y_true, y_pred)
    return ari, v_measure


def calculate_perfect_reconstruction_rate(df, poem_clusters):
    poem_to_type = df.groupby('idoriginal_poem')['type_id'].first().to_dict()

    gt_to_poems = defaultdict(set)
    for poem_id, gt_type in poem_to_type.items():
        gt_to_poems[gt_type].add(poem_id)

    pred_to_poems = defaultdict(set)
    for poem_id, pred_cluster in poem_clusters.items():
        pred_to_poems[pred_cluster].add(poem_id)

    perfectly_reconstructed = 0
    total_gt_clusters = len(gt_to_poems)

    for gt_type, gt_poems in gt_to_poems.items():
        for pred_cluster, pred_poems in pred_to_poems.items():
            if gt_poems == pred_poems:
                perfectly_reconstructed += 1
                break

    reconstruction_rate = perfectly_reconstructed / total_gt_clusters if total_gt_clusters > 0 else 0
    return reconstruction_rate, perfectly_reconstructed, total_gt_clusters


def visualize_verse_grid_search(results_df, save_path=None):
    if save_path is None:
        save_path = RESULTS_DIR / 'verse_grid_search_results.png'

    ari_pivot = results_df.pivot(index='shingle_size', columns='threshold', values='ari')
    vmeasure_pivot = results_df.pivot(index='shingle_size', columns='threshold', values='v_measure')
    clusters_pivot = results_df.pivot(index='shingle_size', columns='threshold', values='n_clusters')
    similarities_pivot = results_df.pivot(index='shingle_size', columns='threshold', values='n_similarities')

    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Verse-Level Clustering Grid Search Results', fontsize=18, fontweight='bold')

    col_labels = [f"{col:.0%}" for col in ari_pivot.columns]

    ax1 = axes[0, 0]
    sns.heatmap(ari_pivot, annot=True, fmt='.4f', cmap='viridis', ax=ax1,
                cbar_kws={'label': 'ARI'}, xticklabels=col_labels)
    ax1.set_xlabel('Similarity Threshold', fontweight='bold', fontsize=12)
    ax1.set_ylabel('Shingle Size', fontweight='bold', fontsize=12)
    ax1.set_title('Adjusted Rand Index (ARI)', fontweight='bold', fontsize=13)

    ax2 = axes[0, 1]
    sns.heatmap(vmeasure_pivot, annot=True, fmt='.4f', cmap='viridis', ax=ax2,
                cbar_kws={'label': 'V-measure'}, xticklabels=col_labels)
    ax2.set_xlabel('Similarity Threshold', fontweight='bold', fontsize=12)
    ax2.set_ylabel('Shingle Size', fontweight='bold', fontsize=12)
    ax2.set_title('V-measure', fontweight='bold', fontsize=13)

    ax3 = axes[1, 0]
    sns.heatmap(clusters_pivot, annot=True, fmt='.0f', cmap='viridis', ax=ax3,
                cbar_kws={'label': 'Clusters'}, xticklabels=col_labels)
    ax3.set_xlabel('Similarity Threshold', fontweight='bold', fontsize=12)
    ax3.set_ylabel('Shingle Size', fontweight='bold', fontsize=12)
    ax3.set_title('Number of Clusters', fontweight='bold', fontsize=13)

    ax4 = axes[1, 1]
    sns.heatmap(similarities_pivot, annot=True, fmt='.0f', cmap='viridis', ax=ax4,
                cbar_kws={'label': 'Similarities'}, xticklabels=col_labels)
    ax4.set_xlabel('Similarity Threshold', fontweight='bold', fontsize=12)
    ax4.set_ylabel('Shingle Size', fontweight='bold', fontsize=12)
    ax4.set_title('Number of Similarities Found', fontweight='bold', fontsize=13)

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"\nVisualization saved to: {save_path}")
    plt.close()


def visualize_poem_grid_search(results_df, save_path=None):
    if save_path is None:
        save_path = RESULTS_DIR / 'poem_grid_search_results.png'
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('Poem-Level Clustering Grid Search Results', fontsize=16, fontweight='bold')

    thresholds = results_df['threshold'].values
    thresholds_pct = [f"{t:.0%}" for t in thresholds]

    def normalize(vals):
        return (vals - np.min(vals)) / (np.max(vals) - np.min(vals))

    ax1 = axes[0, 0]
    norm_vals = normalize(results_df['ari'].values)
    colors = plt.cm.viridis(norm_vals)
    ax1.plot(thresholds_pct, results_df['ari'].values, marker='o', linewidth=2, markersize=8)
    for i, (x, y) in enumerate(zip(thresholds_pct, results_df['ari'].values)):
        ax1.text(i, y, f'{y:.4f}', ha='center', va='bottom', fontsize=9)
    ax1.set_xlabel('Similarity Threshold', fontweight='bold')
    ax1.set_ylabel('Adjusted Rand Index (ARI)', fontweight='bold')
    ax1.set_title('ARI vs Threshold')
    ax1.grid(True, alpha=0.3)

    ax2 = axes[0, 1]
    norm_vals = normalize(results_df['v_measure'].values)
    colors = plt.cm.viridis(norm_vals)
    ax2.plot(thresholds_pct, results_df['v_measure'].values, marker='o', linewidth=2, markersize=8)
    for i, (x, y) in enumerate(zip(thresholds_pct, results_df['v_measure'].values)):
        ax2.text(i, y, f'{y:.4f}', ha='center', va='bottom', fontsize=9)
    ax2.set_xlabel('Similarity Threshold', fontweight='bold')
    ax2.set_ylabel('V-measure', fontweight='bold')
    ax2.set_title('V-measure vs Threshold')
    ax2.grid(True, alpha=0.3)

    ax3 = axes[1, 0]
    prr_vals = results_df['perfect_reconstruction_rate'].values * 100
    norm_vals = normalize(prr_vals)
    colors = plt.cm.viridis(norm_vals)
    ax3.plot(thresholds_pct, prr_vals, marker='o', linewidth=2, markersize=8)
    for i, (x, y) in enumerate(zip(thresholds_pct, prr_vals)):
        ax3.plot(x, y, marker='o', color=colors[i], markersize=10)
        ax3.text(i, y, f'{y:.1f}%', ha='center', va='bottom', fontsize=9)
    ax3.set_xlabel('Similarity Threshold', fontweight='bold')
    ax3.set_ylabel('Perfect Reconstruction Rate (%)', fontweight='bold')
    ax3.set_title('Perfect Reconstruction Rate vs Threshold')
    ax3.grid(True, alpha=0.3)

    ax4 = axes[1, 1]
    n_clusters_vals = results_df['n_clusters'].values
    norm_vals = normalize(n_clusters_vals)
    colors = plt.cm.viridis(norm_vals)
    ax4.plot(thresholds_pct, n_clusters_vals, marker='o', linewidth=2, markersize=8)
    for i, (x, y) in enumerate(zip(thresholds_pct, n_clusters_vals)):
        ax4.plot(x, y, marker='o', color=colors[i], markersize=10)
        ax4.text(i, y, f'{y}', ha='center', va='bottom', fontsize=9)
    ax4.set_xlabel('Similarity Threshold', fontweight='bold')
    ax4.set_ylabel('Number of Poem Clusters', fontweight='bold')
    ax4.set_title('Number of Clusters vs Threshold')
    ax4.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"\nVisualization saved to: {save_path}")
    plt.close()


def verse_level_grid_search(texts, df, thresholds, shingle_sizes, num_perm=128):
    results = []
    best_ari = -1
    best_threshold = None
    best_shingle_size = None
    best_clusters = None
    best_similarities = None

    print("\n" + "="*100)
    print("VERSE-LEVEL 2D GRID SEARCH")
    print("="*100)
    print(f"\nTesting thresholds: {[f'{t:.0%}' for t in thresholds]}")
    print(f"Testing shingle sizes: {shingle_sizes}\n")

    total_combinations = len(thresholds) * len(shingle_sizes)
    print(f"Total combinations: {total_combinations}\n")

    for shingle_size in shingle_sizes:
        for threshold in thresholds:
            print(f"\nTesting shingle_size={shingle_size}, threshold={threshold:.0%}...")

            clusterer = FastMinHashClustering(
                threshold=threshold,
                shingle_size=shingle_size,
                num_perm=num_perm,
                chunk_size=1
            )

            clusters, similarities = clusterer.cluster(texts)

            if 'idgroup' in df.columns:
                temp_df = df.copy()
                temp_df['cluster_id'] = temp_df.index.map(clusters)

                mask = temp_df['idgroup'].notna() & temp_df['cluster_id'].notna()
                y_true = temp_df.loc[mask, 'idgroup'].tolist()
                y_pred = temp_df.loc[mask, 'cluster_id'].tolist()

                ari, v_measure = evaluate_clustering(y_true, y_pred)
                n_gt_clusters = len(set(y_true))
            else:
                ari, v_measure = 0, 0
                n_gt_clusters = 0

            n_clusters = len(set(clusters.values()))

            results.append({
                'shingle_size': shingle_size,
                'threshold': threshold,
                'n_clusters': n_clusters,
                'n_similarities': len(similarities),
                'ari': ari,
                'v_measure': v_measure,
                'n_gt_clusters': n_gt_clusters
            })

            if ari > best_ari:
                best_ari = ari
                best_threshold = threshold
                best_shingle_size = shingle_size
                best_clusters = clusters
                best_similarities = similarities

            print(f"  ARI: {ari:.4f}, V-measure: {v_measure:.4f}, Clusters: {n_clusters}")

    results_df = pd.DataFrame(results)

    print("\n" + "="*100)
    print("VERSE-LEVEL GRID SEARCH SUMMARY")
    print("="*100)
    print(f"\n{'Shingle':<10} {'Threshold':<12} {'Clusters':<10} {'Similarities':<15} {'ARI':<8} {'V-measure':<12}")
    print("-" * 80)

    for _, result in results_df.iterrows():
        print(f"{result['shingle_size']:<10} "
              f"{result['threshold']:<12.0%} "
              f"{result['n_clusters']:<10} "
              f"{result['n_similarities']:<15} "
              f"{result['ari']:<8.4f} "
              f"{result['v_measure']:<12.4f}")

    print(f"\n{'='*100}")
    print("BEST VERSE-LEVEL PARAMETERS")
    print("="*100)
    print(f"\nBest parameters by ARI:")
    print(f"  Shingle size: {best_shingle_size}")
    print(f"  Threshold: {best_threshold:.0%}")
    best_result = results_df[(results_df['threshold'] == best_threshold) &
                              (results_df['shingle_size'] == best_shingle_size)].iloc[0]
    print(f"  ARI: {best_result['ari']:.4f}")
    print(f"  V-measure: {best_result['v_measure']:.4f}")
    print(f"  Number of clusters: {best_result['n_clusters']}")
    print(f"  Number of similarities found: {best_result['n_similarities']}")

    visualize_verse_grid_search(results_df)

    results_csv = RESULTS_DIR / 'verse_grid_search_results.csv'
    results_df.to_csv(results_csv, index=False)
    print(f"\nVerse grid search results saved to: {results_csv}")

    return best_clusters, best_similarities, best_threshold, best_shingle_size, results_df


def poem_level_grid_search(df, poem_to_clusters, thresholds):
    results = []
    best_ari = -1
    best_threshold = None
    best_poem_clusters = None

    print("\n" + "="*100)
    print("POEM-LEVEL GRID SEARCH")
    print("="*100)
    print(f"\nTesting thresholds: {[f'{t:.0%}' for t in thresholds]}\n")

    for threshold in thresholds:
        print(f"\nTesting threshold {threshold:.0%}...")

        poem_clusters, poem_edges, n_clusters = cluster_poems(poem_to_clusters, threshold)

        if 'type_id' in df.columns:
            poem_to_type = df.groupby('idoriginal_poem')['type_id'].first().to_dict()

            y_true = []
            y_pred = []
            for poem_id, predicted_cluster in poem_clusters.items():
                if poem_id in poem_to_type:
                    y_true.append(poem_to_type[poem_id])
                    y_pred.append(predicted_cluster)

            ari, v_measure = evaluate_clustering(y_true, y_pred)
            reconstruction_rate, n_perfect, n_total_gt = calculate_perfect_reconstruction_rate(df, poem_clusters)
        else:
            ari, v_measure = 0, 0
            reconstruction_rate, n_perfect, n_total_gt = 0, 0, 0

        results.append({
            'threshold': threshold,
            'n_clusters': n_clusters,
            'n_edges': len(poem_edges),
            'ari': ari,
            'v_measure': v_measure,
            'perfect_reconstruction_rate': reconstruction_rate,
            'n_perfect_clusters': n_perfect,
            'n_total_gt_clusters': n_total_gt
        })

        if ari > best_ari:
            best_ari = ari
            best_threshold = threshold
            best_poem_clusters = poem_clusters

        print(f"  ARI: {ari:.4f}, V-measure: {v_measure:.4f}, Clusters: {n_clusters}")

    results_df = pd.DataFrame(results)

    print("\n" + "="*100)
    print("POEM-LEVEL GRID SEARCH SUMMARY")
    print("="*100)
    print(f"\n{'Threshold':<12} {'Clusters':<10} {'Edges':<10} {'ARI':<8} {'V-measure':<12} {'Perfect Recon.':<15}")
    print("-" * 80)

    for _, result in results_df.iterrows():
        print(f"{result['threshold']:<12.0%} "
              f"{result['n_clusters']:<10} "
              f"{result['n_edges']:<10} "
              f"{result['ari']:<8.4f} "
              f"{result['v_measure']:<12.4f} "
              f"{result['perfect_reconstruction_rate']:<15.1%}")

    print(f"\n{'='*100}")
    print("BEST POEM-LEVEL THRESHOLD")
    print("="*100)
    print(f"\nBest threshold by ARI: {best_threshold:.0%}")
    best_result = results_df[results_df['threshold'] == best_threshold].iloc[0]
    print(f"  ARI: {best_result['ari']:.4f}")
    print(f"  V-measure: {best_result['v_measure']:.4f}")
    print(f"  Perfect reconstruction rate: {best_result['perfect_reconstruction_rate']:.1%}")
    print(f"    ({best_result['n_perfect_clusters']:.0f}/{best_result['n_total_gt_clusters']:.0f} GT clusters perfectly reconstructed)")

    visualize_poem_grid_search(results_df)

    results_csv = RESULTS_DIR / 'poem_grid_search_results.csv'
    results_df.to_csv(results_csv, index=False)
    print(f"\nPoem grid search results saved to: {results_csv}")

    return best_poem_clusters, best_threshold, results_df


def main():
    DATA_FILE = 'dbbe_full.csv'

    print("="*100)
    print("LOADING DATA")
    print("="*100)
    print(f"Results will be saved to: {RESULTS_DIR}")

    df = pd.read_csv(DATA_FILE)

    if 'verse' in df.columns:
        df['text'] = df['verse']
    elif 'text' not in df.columns:
        raise ValueError("Dataset must have either 'verse' or 'text' column")

    df['text'] = df['text'].fillna('').astype(str)
    print(f"\nLoaded {df.shape[0]:,} verses")

    texts = df['text'].tolist()

    verse_thresholds = [0.2, 0.3, 0.4, 0.5]
    shingle_sizes = [2, 3, 4, 5]

    best_clusters, best_similarities, best_verse_threshold, best_shingle_size, verse_results = verse_level_grid_search(
        texts, df, verse_thresholds, shingle_sizes, num_perm=128
    )

    df['cluster_id'] = df.index.map(best_clusters)

    sim_dict = defaultdict(list)
    for doc1, doc2, sim in best_similarities:
        sim_dict[doc1].append(sim)
        sim_dict[doc2].append(sim)

    df['certainty'] = df.index.map(
        lambda i: np.mean(sim_dict[i]) if i in sim_dict else 1.0
    )

    preprocessor = TextPreprocessor(lowercase=True, remove_punctuation=True, remove_diacritics=True)
    df['text_preprocessed'] = df['text'].apply(preprocessor.preprocess)

    verse_output = RESULTS_DIR / "dbbe_verse_clustered_results.csv"
    df.to_csv(verse_output, index=False)
    print(f"\n{verse_output} saved with best parameters (shingle_size={best_shingle_size}, threshold={best_verse_threshold:.0%})")

    if 'idoriginal_poem' in df.columns and 'type_id' in df.columns:
        poem_to_clusters, poem_verse_counts = reconstruct_poems(df)

        poem_thresholds = [0.50, 0.60, 0.70, 0.8]

        best_poem_clusters, best_poem_threshold, poem_results = poem_level_grid_search(
            df, poem_to_clusters, poem_thresholds
        )

        df['poem_cluster_id'] = df['idoriginal_poem'].map(best_poem_clusters)

        poem_output = RESULTS_DIR / "dbbe_poem_level_clusters.csv"
        df.to_csv(poem_output, index=False)
        print(f"\n{poem_output} saved with best threshold ({best_poem_threshold:.0%})")
    else:
        print("\n" + "="*100)
        print("SKIPPING POEM-LEVEL CLUSTERING")
        print("="*100)
        print("\nRequired columns 'idoriginal_poem' and/or 'type_id' not found")

    print("\n" + "="*100)
    print("ANALYSIS COMPLETE")
    print("="*100)
    print(f"All results saved to: {RESULTS_DIR}")


if __name__ == "__main__":
    main()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path
%matplotlib inline
RESULTS_DIR = Path("dbbe_orthographic_results")

results_df = pd.read_csv(RESULTS_DIR / 'verse_grid_search_results.csv')


def visualize_verse_grid_search_larger_font(results_df, save_path=None):
    if save_path is None:
        save_path = RESULTS_DIR / 'verse_grid_search_results_large_font.png'

    ari_pivot = results_df.pivot(index='shingle_size', columns='threshold', values='ari')
    vmeasure_pivot = results_df.pivot(index='shingle_size', columns='threshold', values='v_measure')
    clusters_pivot = results_df.pivot(index='shingle_size', columns='threshold', values='n_clusters')
    similarities_pivot = results_df.pivot(index='shingle_size', columns='threshold', values='n_similarities')

    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Verse-Level Clustering Grid Search Results', fontsize=22, fontweight='bold')  # larger

    col_labels = [f"{col:.0%}" for col in ari_pivot.columns]

    for ax, pivot, title, cbar_label in zip(
        axes.flat,
        [ari_pivot, vmeasure_pivot, clusters_pivot, similarities_pivot],
        ['Adjusted Rand Index (ARI)', 'V-measure', 'Number of Clusters', 'Number of Similarities Found'],
        ['ARI', 'V-measure', 'Clusters', 'Similarities']
    ):
        sns.heatmap(pivot, annot=True, fmt='.4f' if pivot is not clusters_pivot and pivot is not similarities_pivot else '.0f',
                    cmap='viridis', ax=ax, cbar_kws={'label': cbar_label}, annot_kws={'fontsize':16})
        ax.set_xlabel('Similarity Threshold', fontsize=14, fontweight='bold')
        ax.set_ylabel('Shingle Size', fontsize=14, fontweight='bold')
        ax.set_title(title, fontsize=16, fontweight='bold')

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
visualize_verse_grid_search_larger_font(results_df)

In [25]:
import pandas as pd
from collections import defaultdict
from pathlib import Path

# Load CSV
df = pd.read_csv('dbbe_orthographic_results/dbbe_poem_level_clusters.csv')
output_file = Path("dbbe_orthographic_results/five_largest_clusters.txt")

# Reconstruct poems: poem_id -> list of (order, text)
poem_to_verses = defaultdict(list)
for _, row in df.iterrows():
    poem_id = row['idoriginal_poem']
    order = row['order'] if 'order' in df.columns else 0
    text = str(row['text']) if pd.notna(row['text']) else ''
    poem_to_verses[poem_id].append((order, text))

# Cluster -> list of poem_ids
cluster_to_poems = defaultdict(list)
for _, row in df.iterrows():
    cid = row['poem_cluster_id']
    poem_id = row['idoriginal_poem']
    if poem_id not in cluster_to_poems[cid]:
        cluster_to_poems[cid].append(poem_id)

# Compute cluster sizes by number of poems
cluster_sizes = {cid: len(pids) for cid, pids in cluster_to_poems.items()}

# Get five largest clusters
top_clusters = sorted(cluster_sizes.items(), key=lambda x: x[1], reverse=True)[:5]

# Write output
with open(output_file, 'w', encoding='utf-8') as f:
    for cid, size in top_clusters:
        f.write(f"CLUSTER ID={cid}, poems={size}\n")
        f.write("-" * 80 + "\n")
        for poem_id in cluster_to_poems[cid]:
            # Sort verses in original order
            verses_sorted = [v for _, v in sorted(poem_to_verses[poem_id], key=lambda x: x[0])]
            f.write(f"Poem ID: {poem_id}\n")
            for verse_text in verses_sorted:
                # mark verse with asterisk if it's in this cluster
                rows = df[(df['idoriginal_poem'] == poem_id) & (df['text'] == verse_text)]
                prefix = "* " if (rows['poem_cluster_id'] == cid).any() else "  "
                f.write(f"{prefix}{verse_text}\n")
            f.write("-" * 40 + "\n")
        f.write("\n")

print(f"Five largest clusters saved to: {output_file}")


Five largest clusters saved to: dbbe_orthographic_results/five_largest_clusters.txt


# 2. Full dataset

## 2.1 Verse level clustering

In [28]:
import re
import unicodedata
from typing import Dict
import pandas as pd
import numpy as np
from tqdm import tqdm
from datasketch import MinHash, MinHashLSHForest
import multiprocessing as mp
import hashlib
import time
import matplotlib.pyplot as plt
import seaborn as sns
import os
from concurrent.futures import ProcessPoolExecutor, as_completed

CLEAN_PATTERN = re.compile(r'[^\w\s]')
WHITESPACE_PATTERN = re.compile(r'\s+')

def preprocess_text(text: str, options: Dict[str, bool] = None) -> str:
    if options is None:
        options = {'lowercase': True, 'remove_diacritics': True, 'remove_punctuation': True}
    text = str(text)
    if options.get('lowercase', True):
        text = text.lower()
    if options.get('remove_diacritics', True):
        text = unicodedata.normalize('NFD', text)
        text = ''.join(char for char in text if unicodedata.category(char) != 'Mn')
        text = unicodedata.normalize('NFC', text)
    else:
        text = unicodedata.normalize('NFC', text)
    if options.get('remove_punctuation', True):
        text = CLEAN_PATTERN.sub('', text)
    text = WHITESPACE_PATTERN.sub(' ', text)
    return text.strip()

class FastUnionFind:
    __slots__ = ['parent', 'rank']
    
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n
    
    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    
    def union(self, x, y):
        px, py = self.find(x), self.find(y)
        if px == py:
            return False
        if self.rank[px] < self.rank[py]:
            self.parent[px] = py
        elif self.rank[px] > self.rank[py]:
            self.parent[py] = px
        else:
            self.parent[py] = px
            self.rank[px] += 1
        return True
    
    def get_clusters(self):
        return np.array([self.find(i) for i in range(len(self.parent))], dtype=np.int32)

def get_ngrams_vectorized(text, n=4):
    if not text or len(text) < n:
        return set()
    text = str(text).lower()
    return set(text[i:i+n] for i in range(len(text)-n+1))

def compute_minhash_chunk(args):
    texts, start_idx, n_gram_size, num_perm, seed = args
    np.random.seed(seed)
    minhashes = []
    for text in texts:
        ngrams = get_ngrams_vectorized(text, n_gram_size)
        m = MinHash(num_perm=num_perm, seed=seed)
        if ngrams:
            for ngram in ngrams:
                m.update(ngram.encode('utf8'))
        minhashes.append(m)
    return minhashes

def compute_minhash_parallel(texts, n_gram_size=3, num_perm=128, n_cores=None):
    if n_cores is None:
        n_cores = min(mp.cpu_count(), 16)
    chunk_size = max(100, min(2000, len(texts) // (n_cores * 2)))
    chunks = [(texts[i:i+chunk_size], i, n_gram_size, num_perm, 42) 
              for i in range(0, len(texts), chunk_size)]
    with mp.Pool(n_cores) as pool:
        results = list(tqdm(pool.imap(compute_minhash_chunk, chunks),
                          total=len(chunks), desc=f"MinHash (n={n_gram_size})", leave=False))
    minhashes = [mh for chunk_mhs in results for mh in chunk_mhs]
    return minhashes

def fast_hash(data):
    return int(hashlib.md5(data).hexdigest()[:16], 16)

def find_exact_duplicates_fast(texts):
    text_hashes = {}
    for i, text in enumerate(tqdm(texts, desc="Hashing", disable=len(texts)<100000, leave=False)):
        normalized = str(text).strip().lower()
        if not normalized:
            continue
        text_hash = fast_hash(normalized.encode('utf-8'))
        text_hashes.setdefault(text_hash, []).append(i)
    duplicate_groups = [indices for indices in text_hashes.values() if len(indices) > 1]
    return duplicate_groups

def stratified_sample(df, n_sample=15000):
    datasets = df['source_dataset'].unique()
    total_size = len(df)
    sample_indices = []
    
    for dataset in datasets:
        dataset_indices = df[df['source_dataset'] == dataset].index.tolist()
        dataset_size = len(dataset_indices)
        proportion = dataset_size / total_size
        n_from_dataset = int(n_sample * proportion)
        n_from_dataset = min(n_from_dataset, dataset_size)
        if n_from_dataset > 0:
            sampled = np.random.choice(dataset_indices, size=n_from_dataset, replace=False)
            sample_indices.extend(sampled)
    
    return sorted(sample_indices)

def compute_cluster_cohesion(minhashes, cluster_labels):
    """
    Compute average within-cluster similarity (cohesion).
    Higher is better - indicates tight clusters.
    """
    unique_clusters = np.unique(cluster_labels)
    cohesions = []
    
    for cluster_id in unique_clusters:
        cluster_indices = np.where(cluster_labels == cluster_id)[0]
        if len(cluster_indices) < 2:
            continue
        
        # Sample pairs if cluster is too large
        if len(cluster_indices) > 50:
            sampled_indices = np.random.choice(cluster_indices, 50, replace=False)
        else:
            sampled_indices = cluster_indices
        
        # Compute average pairwise similarity
        sims = []
        for i in range(len(sampled_indices)):
            for j in range(i+1, len(sampled_indices)):
                sim = minhashes[sampled_indices[i]].jaccard(minhashes[sampled_indices[j]])
                sims.append(sim)
        
        if sims:
            cohesions.append(np.mean(sims))
    
    return np.mean(cohesions) if cohesions else 0.0

def compute_cluster_separation(minhashes, cluster_labels, n_samples=500):
    """
    Compute average between-cluster dissimilarity (separation).
    Higher is better - indicates well-separated clusters.
    """
    unique_clusters = np.unique(cluster_labels)
    if len(unique_clusters) < 2:
        return 1.0  # Perfect separation if only one cluster
    
    # Sample pairs from different clusters
    separations = []
    for _ in range(n_samples):
        # Sample two different clusters
        c1, c2 = np.random.choice(unique_clusters, 2, replace=False)
        
        # Sample one point from each cluster
        idx1 = np.random.choice(np.where(cluster_labels == c1)[0])
        idx2 = np.random.choice(np.where(cluster_labels == c2)[0])
        
        # Compute dissimilarity (1 - similarity)
        sim = minhashes[idx1].jaccard(minhashes[idx2])
        separations.append(1 - sim)
    
    return np.mean(separations) if separations else 0.0

def compute_silhouette_approximation(minhashes, cluster_labels, n_samples=1000):
    """
    Approximate silhouette score using sampling.
    Ranges from -1 to 1, higher is better.
    """
    unique_clusters = np.unique(cluster_labels)
    if len(unique_clusters) < 2:
        return 0.0
    
    # Sample points for efficiency
    n_total = len(cluster_labels)
    if n_total > n_samples:
        sample_indices = np.random.choice(n_total, n_samples, replace=False)
    else:
        sample_indices = np.arange(n_total)
    
    silhouettes = []
    
    for idx in sample_indices:
        cluster_id = cluster_labels[idx]
        
        # Compute average distance to points in same cluster (a)
        same_cluster = np.where(cluster_labels == cluster_id)[0]
        same_cluster = same_cluster[same_cluster != idx]
        
        if len(same_cluster) == 0:
            continue
        
        # Sample if too many points
        if len(same_cluster) > 20:
            same_cluster = np.random.choice(same_cluster, 20, replace=False)
        
        a = np.mean([1 - minhashes[idx].jaccard(minhashes[j]) 
                    for j in same_cluster])
        
        # Compute average distance to nearest other cluster (b)
        other_clusters = unique_clusters[unique_clusters != cluster_id]
        if len(other_clusters) == 0:
            continue
        
        min_b = float('inf')
        for other_id in other_clusters:
            other_cluster = np.where(cluster_labels == other_id)[0]
            
            # Sample if too many points
            if len(other_cluster) > 20:
                other_cluster = np.random.choice(other_cluster, 20, replace=False)
            
            b = np.mean([1 - minhashes[idx].jaccard(minhashes[j]) 
                        for j in other_cluster])
            min_b = min(min_b, b)
        
        # Silhouette coefficient
        s = (min_b - a) / max(a, min_b) if max(a, min_b) > 0 else 0
        silhouettes.append(s)
    
    return np.mean(silhouettes) if silhouettes else 0.0

def evaluate_single_config(args):
    """
    Evaluate a single (shingle_size, threshold) configuration.
    Returns metrics for quality assessment.
    """
    shingle_size, threshold, texts, sample_indices, duplicate_groups = args
    
    try:
        # Compute MinHashes for this shingle size
        sample_texts = [texts[i] for i in sample_indices]
        minhashes_sample = compute_minhash_parallel(
            sample_texts, 
            n_gram_size=shingle_size, 
            num_perm=128
        )
        
        # Build LSH Forest
        forest = MinHashLSHForest(num_perm=128)
        for idx, mh in enumerate(minhashes_sample):
            forest.add(str(idx), mh)
        forest.index()
        
        # Cluster using Union-Find
        n_sample = len(sample_indices)
        uf = FastUnionFind(n_sample)
        
        # Add exact duplicates from the sample
        sample_set = set(sample_indices)
        for group in duplicate_groups:
            sample_group = [sample_indices.index(g) for g in group if g in sample_set]
            if len(sample_group) > 1:
                for i in range(1, len(sample_group)):
                    uf.union(sample_group[0], sample_group[i])
        
        # Query and merge similar pairs
        top_k = 50
        merges = 0
        for idx in range(n_sample):
            if uf.find(idx) != idx:
                continue
            neighbors = forest.query(minhashes_sample[idx], top_k)
            for neighbor_str in neighbors[1:]:
                neighbor_idx = int(neighbor_str)
                if uf.find(idx) == uf.find(neighbor_idx):
                    continue
                sim = minhashes_sample[idx].jaccard(minhashes_sample[neighbor_idx])
                if sim >= threshold:
                    if uf.union(idx, neighbor_idx):
                        merges += 1
        
        cluster_labels = uf.get_clusters()
        unique_clusters, cluster_sizes = np.unique(cluster_labels, return_counts=True)
        
        # Compute quality metrics
        n_clusters = len(unique_clusters)
        n_multi = np.sum(cluster_sizes > 1)
        n_singleton = np.sum(cluster_sizes == 1)
        avg_size = float(cluster_sizes.mean())
        max_size = int(cluster_sizes.max())
        
        # Advanced metrics
        cohesion = compute_cluster_cohesion(minhashes_sample, cluster_labels)
        separation = compute_cluster_separation(minhashes_sample, cluster_labels)
        silhouette = compute_silhouette_approximation(minhashes_sample, cluster_labels)
        
        return {
            'shingle_size': shingle_size,
            'threshold': threshold,
            'n_clusters': n_clusters,
            'n_multi_clusters': n_multi,
            'n_singletons': n_singleton,
            'avg_cluster_size': avg_size,
            'max_cluster_size': max_size,
            'cohesion': cohesion,
            'separation': separation,
            'silhouette': silhouette,
            'merges': merges
        }
    
    except Exception as e:
        print(f"Error at shingle={shingle_size}, threshold={threshold:.2f}: {e}")
        return None

def grid_search_parameters(texts, df, duplicate_groups, 
                          shingle_sizes=[2, 3, 4, 5],
                          threshold_range=(0.3, 0.9, 7),
                          n_sample=15000,
                          results_folder="full_orthographic_results",
                          max_workers=8):
    """
    Perform 2D grid search over shingle size and threshold.
    
    Selection Metric: Combined Quality Score
    - Silhouette score (40%): Balance between cohesion and separation
    - Cohesion (30%): Within-cluster similarity
    - Separation (20%): Between-cluster dissimilarity
    - Cluster balance (10%): Penalize too many singletons
    """
    print("\n" + "="*80)
    print("2D GRID SEARCH: SHINGLE SIZE × THRESHOLD")
    print("="*80)
    
    # Sample for evaluation
    sample_indices = stratified_sample(df, n_sample)
    print(f"Sample size: {len(sample_indices):,}")
    
    # Generate threshold grid
    thresholds = np.linspace(threshold_range[0], threshold_range[1], threshold_range[2])
    
    print(f"\nParameter grid:")
    print(f"  Shingle sizes: {shingle_sizes}")
    print(f"  Thresholds: {len(thresholds)} values from {thresholds[0]:.2f} to {thresholds[-1]:.2f}")
    print(f"  Total combinations: {len(shingle_sizes) * len(thresholds)}")
    
    # Prepare arguments for parallel evaluation
    args_list = []
    for shingle_size in shingle_sizes:
        for threshold in thresholds:
            args_list.append((shingle_size, threshold, texts, sample_indices, duplicate_groups))
    
    # Parallel evaluation
    print(f"\nRunning grid search with {max_workers} workers...")
    start_time = time.time()
    
    results = []
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(evaluate_single_config, args): args for args in args_list}
        
        with tqdm(total=len(futures), desc="Grid search") as pbar:
            for future in as_completed(futures):
                result = future.result()
                if result is not None:
                    results.append(result)
                pbar.update(1)
    
    print(f"✓ Grid search complete in {time.time()-start_time:.1f}s")
    print(f"  Valid results: {len(results)} / {len(args_list)}")
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    
    # Compute combined quality score
    print("\nComputing quality scores...")
    
    # Normalize metrics to 0-1 range
    def normalize(series):
        min_val = series.min()
        max_val = series.max()
        if max_val - min_val < 1e-10:
            return pd.Series(0.5, index=series.index)
        return (series - min_val) / (max_val - min_val)
    
    silhouette_score = normalize(results_df['silhouette'])
    cohesion_score = normalize(results_df['cohesion'])
    separation_score = normalize(results_df['separation'])
    
    # Cluster balance: prefer moderate number of clusters, penalize too many singletons
    singleton_ratio = results_df['n_singletons'] / len(sample_indices)
    balance_score = 1 - singleton_ratio
    balance_score = np.clip(balance_score, 0, 1)
    
    # Combined score with weights
    results_df['quality_score'] = (
        silhouette_score * 0.40 +
        cohesion_score * 0.30 +
        separation_score * 0.20 +
        balance_score * 0.10
    )
    
    # Save results
    results_df = results_df.sort_values('quality_score', ascending=False)
    results_csv = os.path.join(results_folder, 'parameter_grid_search_results.csv')
    results_df.to_csv(results_csv, index=False)
    print(f"✓ Results saved: {results_csv}")
    
    # Visualizations
    create_grid_search_visualizations(results_df, results_folder)
    
    # Select best configuration
    best_config = results_df.iloc[0]
    best_shingle = int(best_config['shingle_size'])
    best_threshold = float(best_config['threshold'])
    
    print("\n" + "="*80)
    print("TOP 5 CONFIGURATIONS (BY QUALITY SCORE)")
    print("="*80)
    
    for idx, (i, row) in enumerate(results_df.head(5).iterrows(), 1):
        print(f"\n#{idx}. Shingle size: {int(row['shingle_size'])}, Threshold: {row['threshold']:.3f}")
        print(f"     Quality score: {row['quality_score']:.3f}")
        print(f"     Silhouette: {row['silhouette']:.3f}, Cohesion: {row['cohesion']:.3f}, "
              f"Separation: {row['separation']:.3f}")
        print(f"     Clusters: {int(row['n_multi_clusters']):,}, Singletons: {int(row['n_singletons']):,}, "
              f"Avg size: {row['avg_cluster_size']:.1f}")
    
    print("\n" + "="*80)
    print("🎯 SELECTED CONFIGURATION (HIGHEST QUALITY)")
    print("="*80)
    print(f"Shingle size: {best_shingle}")
    print(f"Threshold: {best_threshold:.3f}")
    print(f"Quality score: {best_config['quality_score']:.3f}")
    print(f"  - Silhouette: {best_config['silhouette']:.3f}")
    print(f"  - Cohesion: {best_config['cohesion']:.3f}")
    print(f"  - Separation: {best_config['separation']:.3f}")
    print(f"Multi-member clusters: {int(best_config['n_multi_clusters']):,}")
    print(f"Singletons: {int(best_config['n_singletons']):,}")
    print("="*80)
    
    return best_shingle, best_threshold, results_df

def create_grid_search_visualizations(results_df, results_folder):
    """Create comprehensive visualizations of the grid search results."""
    print("\nCreating visualizations...")
    
    sns.set_palette("colorblind")
    fig = plt.figure(figsize=(18, 12))
    gs = fig.add_gridspec(3, 3, hspace=0.35, wspace=0.3)
    
    # 1. Quality score heatmap
    ax1 = fig.add_subplot(gs[0, :2])
    pivot_quality = results_df.pivot_table(
        values='quality_score',
        index='threshold',
        columns='shingle_size',
        aggfunc='first'
    )
    sns.heatmap(pivot_quality, annot=True, fmt='.3f', cmap='RdYlGn', ax=ax1,
               cbar_kws={'label': 'Quality Score'})
    ax1.set_ylabel('Threshold', fontweight='bold')
    ax1.set_xlabel('Shingle Size', fontweight='bold')
    ax1.set_title('Quality Score Heatmap', fontweight='bold', fontsize=14)
    
    # Mark best combination
    best = results_df.iloc[0]
    if best['shingle_size'] in pivot_quality.columns and best['threshold'] in pivot_quality.index:
        best_col = list(pivot_quality.columns).index(best['shingle_size'])
        best_row = list(pivot_quality.index).index(best['threshold'])
        ax1.add_patch(plt.Rectangle((best_col, best_row), 1, 1,
                                   fill=False, edgecolor='blue', lw=3))
    
    # 2. Quality score distribution
    ax2 = fig.add_subplot(gs[0, 2])
    ax2.hist(results_df['quality_score'], bins=20, color='#0173B2', alpha=0.7, edgecolor='black')
    ax2.axvline(best['quality_score'], color='red', linestyle='--', linewidth=2,
               label=f"Best: {best['quality_score']:.3f}")
    ax2.set_xlabel('Quality Score', fontweight='bold')
    ax2.set_ylabel('Frequency', fontweight='bold')
    ax2.set_title('Score Distribution', fontweight='bold')
    ax2.legend()
    ax2.grid(alpha=0.3)
    
    # 3. Silhouette heatmap
    ax3 = fig.add_subplot(gs[1, 0])
    pivot_silhouette = results_df.pivot_table(
        values='silhouette',
        index='threshold',
        columns='shingle_size',
        aggfunc='first'
    )
    sns.heatmap(pivot_silhouette, annot=True, fmt='.3f', cmap='coolwarm', ax=ax3,
               cbar_kws={'label': 'Silhouette'})
    ax3.set_ylabel('Threshold', fontweight='bold')
    ax3.set_xlabel('Shingle Size', fontweight='bold')
    ax3.set_title('Silhouette Score', fontweight='bold', fontsize=12)
    
    # 4. Cohesion heatmap
    ax4 = fig.add_subplot(gs[1, 1])
    pivot_cohesion = results_df.pivot_table(
        values='cohesion',
        index='threshold',
        columns='shingle_size',
        aggfunc='first'
    )
    sns.heatmap(pivot_cohesion, annot=True, fmt='.3f', cmap='YlOrRd', ax=ax4,
               cbar_kws={'label': 'Cohesion'})
    ax4.set_ylabel('Threshold', fontweight='bold')
    ax4.set_xlabel('Shingle Size', fontweight='bold')
    ax4.set_title('Within-Cluster Cohesion', fontweight='bold', fontsize=12)
    
    # 5. Separation heatmap
    ax5 = fig.add_subplot(gs[1, 2])
    pivot_separation = results_df.pivot_table(
        values='separation',
        index='threshold',
        columns='shingle_size',
        aggfunc='first'
    )
    sns.heatmap(pivot_separation, annot=True, fmt='.3f', cmap='YlGnBu', ax=ax5,
               cbar_kws={'label': 'Separation'})
    ax5.set_ylabel('Threshold', fontweight='bold')
    ax5.set_xlabel('Shingle Size', fontweight='bold')
    ax5.set_title('Between-Cluster Separation', fontweight='bold', fontsize=12)
    
    # 6. Quality vs threshold for different shingle sizes
    ax6 = fig.add_subplot(gs[2, 0])
    for shingle in sorted(results_df['shingle_size'].unique()):
        data = results_df[results_df['shingle_size'] == shingle]
        ax6.plot(data['threshold'], data['quality_score'], 'o-',
                label=f'n={int(shingle)}', linewidth=2, markersize=5, alpha=0.7)
    ax6.axhline(best['quality_score'], color='red', linestyle='--',
               linewidth=1, alpha=0.5, label=f'Best: {best["quality_score"]:.3f}')
    ax6.set_xlabel('Threshold', fontweight='bold')
    ax6.set_ylabel('Quality Score', fontweight='bold')
    ax6.set_title('Quality vs Threshold', fontweight='bold')
    ax6.legend(fontsize=8)
    ax6.grid(alpha=0.3)
    
    # 7. Number of clusters heatmap
    ax7 = fig.add_subplot(gs[2, 1])
    pivot_clusters = results_df.pivot_table(
        values='n_multi_clusters',
        index='threshold',
        columns='shingle_size',
        aggfunc='first'
    )
    sns.heatmap(pivot_clusters, annot=True, fmt='.0f', cmap='viridis', ax=ax7,
               cbar_kws={'label': 'Multi-Clusters'})
    ax7.set_ylabel('Threshold', fontweight='bold')
    ax7.set_xlabel('Shingle Size', fontweight='bold')
    ax7.set_title('Multi-Member Clusters', fontweight='bold', fontsize=12)
    
    # 8. Silhouette vs Cohesion scatter
    ax8 = fig.add_subplot(gs[2, 2])
    scatter = ax8.scatter(results_df['cohesion'], results_df['silhouette'],
                         c=results_df['quality_score'], cmap='RdYlGn',
                         s=80, alpha=0.7, edgecolors='black', linewidth=0.5)
    ax8.scatter(best['cohesion'], best['silhouette'],
               color='red', s=250, marker='*', edgecolors='black', linewidth=2,
               label='Best', zorder=10)
    ax8.set_xlabel('Cohesion', fontweight='bold')
    ax8.set_ylabel('Silhouette', fontweight='bold')
    ax8.set_title('Silhouette vs Cohesion', fontweight='bold')
    plt.colorbar(scatter, ax=ax8, label='Quality Score')
    ax8.legend()
    ax8.grid(alpha=0.3)
    
    fig.suptitle(f'2D Grid Search: Shingle Size × Threshold (n={len(results_df)} configurations)',
                fontsize=16, fontweight='bold', y=0.995)
    
    plt.tight_layout()
    plot_path = os.path.join(results_folder, 'grid_search_comprehensive.png')
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    print(f"✓ Visualization saved: {plot_path}")
    plt.close()

def cluster_with_lsh_forest(minhashes, duplicate_groups, threshold, top_k=100):
    n_docs = len(minhashes)
    uf = FastUnionFind(n_docs)
    
    exact_merges = 0
    for group in duplicate_groups:
        for i in range(1, len(group)):
            if uf.union(group[0], group[i]):
                exact_merges += 1
    
    forest = MinHashLSHForest(num_perm=len(minhashes[0].hashvalues))
    for idx, mh in enumerate(tqdm(minhashes, desc="Indexing", leave=False)):
        forest.add(str(idx), mh)
    forest.index()
    
    lsh_merges = 0
    verified_pairs = 0
    chunk_size = 10000
    
    for start_idx in tqdm(range(0, n_docs, chunk_size), desc="Clustering"):
        end_idx = min(start_idx + chunk_size, n_docs)
        for idx in range(start_idx, end_idx):
            if uf.find(idx) != idx:
                continue
            neighbors = forest.query(minhashes[idx], top_k)
            for neighbor_str in neighbors[1:]:
                neighbor_idx = int(neighbor_str)
                if uf.find(idx) == uf.find(neighbor_idx):
                    continue
                verified_pairs += 1
                sim = minhashes[idx].jaccard(minhashes[neighbor_idx])
                if sim >= threshold:
                    if uf.union(idx, neighbor_idx):
                        lsh_merges += 1
    
    cluster_labels = uf.get_clusters()
    unique_clusters, cluster_sizes = np.unique(cluster_labels, return_counts=True)
    
    return cluster_labels, {
        'n_clusters': len(unique_clusters),
        'n_multi_clusters': np.sum(cluster_sizes > 1),
        'n_singletons': np.sum(cluster_sizes == 1),
        'avg_cluster_size': float(cluster_sizes.mean()),
        'max_cluster_size': int(cluster_sizes.max()),
        'exact_merges': exact_merges,
        'lsh_merges': lsh_merges,
        'threshold': threshold,
        'verified_pairs': verified_pairs
    }

def main():
    # Create results folder
    results_folder = "full_orthographic_results"
    os.makedirs(results_folder, exist_ok=True)
    print(f"Results will be saved to: {results_folder}/\n")
    
    # Load data
    df = pd.read_csv("concatenated.csv")
    df = df[df['source_dataset'].isin(['rhoby', 'dbbe', 'phi', 'papyri'])]
    df = df[df['verse'].fillna('').astype(str).str.len() >= 20]
    df['verse'] = df['verse'].apply(preprocess_text)
    df = df.reset_index(drop=True)
    df = df[df['verse'].str.strip().str.lower() != 'nan']
    texts = df['verse'].fillna('').astype(str).tolist()
    
    print(f"Verses: {len(texts):,}")
    
    start_time = time.time()
    
    # Find exact duplicates (fast, independent of parameters)
    duplicate_groups = find_exact_duplicates_fast(texts)
    t1 = time.time()
    print(f"✓ Found {len(duplicate_groups):,} exact duplicate groups in {t1-start_time:.1f}s")
    
    # 2D Grid search for optimal shingle size and threshold
    best_shingle, best_threshold, grid_results = grid_search_parameters(
        texts, df, duplicate_groups,
        shingle_sizes=[2, 3, 4, 5],
        threshold_range=(0.3, 0.85, 7),
        n_sample=15000,
        results_folder=results_folder,
        max_workers=8
    )
    t2 = time.time()
    
    # Compute MinHashes with optimal shingle size
    print(f"\nComputing MinHashes with optimal shingle size={best_shingle}...")
    minhashes = compute_minhash_parallel(texts, n_gram_size=best_shingle, num_perm=128)
    t3 = time.time()
    
    # Cluster with optimal parameters
    print(f"\nClustering with threshold={best_threshold:.3f}...")
    cluster_labels, metrics = cluster_with_lsh_forest(
        minhashes, duplicate_groups, best_threshold, top_k=100
    )
    t4 = time.time()
    
    total_time = time.time() - start_time
    
    # Save results
    df['cluster_id'] = cluster_labels
    output_csv = os.path.join(results_folder, "clustered_optimized.csv")
    df.to_csv(output_csv, index=False)
    print(f"✓ Clustered data saved: {output_csv}")
    
    # Save metrics
    metrics.update({
        'total_time_minutes': total_time / 60,
        'best_shingle_size': best_shingle,
        'best_threshold': best_threshold,
        'grid_search_time_seconds': t2 - t1,
        'minhash_time_seconds': t3 - t2,
        'clustering_time_seconds': t4 - t3
    })
    metrics_csv = os.path.join(results_folder, "clustering_metrics.csv")
    pd.DataFrame([metrics]).to_csv(metrics_csv, index=False)
    print(f"✓ Metrics saved: {metrics_csv}")
    
    print(f"\n{'='*70}")
    print("CLUSTERING COMPLETE")
    print(f"{'='*70}")
    print(f"Total time: {total_time/60:.1f} min")
    print(f"Optimal shingle size: {best_shingle}")
    print(f"Optimal threshold: {best_threshold:.3f}")
    print(f"Multi-member clusters: {metrics['n_multi_clusters']:,}")
    print(f"Singletons: {metrics['n_singletons']:,}")
    print(f"Max cluster size: {metrics['max_cluster_size']}")
    print(f"All results saved to: {results_folder}/")
    print(f"{'='*70}")

if __name__ == "__main__":
    main()

Results will be saved to: full_orthographic_results/



/tmp/ipykernel_306607/1585915595.py:245: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("concatenated.csv")


Verses: 1,537,740


MinHash: 100%|██████████| 769/769 [03:38<00:00,  3.52it/s]


Threshold selection plot saved to: full_orthographic_results/threshold_selection.png
Threshold: 0.300


Clustering: 100%|██████████| 154/154 [14:12<00:00,  5.53s/it]         


Clustered data saved to: full_orthographic_results/clustered_ultrafast.csv
Metrics saved to: full_orthographic_results/clustering_metrics.csv

CLUSTERING COMPLETE
Time: 19.1 min
Clusters (multi-member): 58,639
Max cluster size: 1083656
All results saved to: full_orthographic_results/


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Load the CSV
df = pd.read_csv('full_orthographic_results/parameter_grid_search_results.csv')

# Pivot the data
pivot = df.pivot(index='threshold', columns='shingle_size', values='quality_score')

# Ensure index is float and rounded
pivot.index = pd.Index(np.round(pivot.index.astype(float), 2))

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(
    pivot, 
    annot=True, 
    fmt=".2f", 
    cmap='viridis', 
    cbar_kws={'label': 'Quality Score'},
    annot_kws={"size": 22}  # increase annotation font size
)

plt.title('Grid Search: Shingle Size × Threshold', fontsize=18, fontweight='bold')
plt.xlabel('Shingle Size', fontsize=14, fontweight='bold')
plt.ylabel('Threshold', fontsize=14, fontweight='bold')

# Round y-axis tick labels and increase font size
plt.yticks(rotation=0, fontsize=12)
plt.xticks(fontsize=12)

plt.gca().set_yticklabels([f"{tick:.2f}" for tick in pivot.index])
%matplotlib inline
plt.tight_layout()
plt.savefig('full_orthographic_results/parameter_grid_search.png', dpi=300, bbox_inches='tight')
plt.show()


## 2.2 Poem level clustering

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
from numba import njit
import time


INPUT_CSV = "full_orthographic_results/clustered_optimized.csv"

SAMPLE_SIZE = 15000  
RANDOM_SEED = 42
MIN_SHARED_VERSES = 2

# Grid search parameters for testing multiple thresholds
THRESHOLD_RANGE = (0.3, 0.9, 13)  # (min, max, n_points)


class EnhancedPoemThresholdSelector:
    """
    Analyze poem-level similarity distribution and select threshold
    based on quality metrics (silhouette, cohesion, separation)
    """
    
    def __init__(self, sample_size: int = 10000, random_seed: int = 42, min_shared_verses: int = 2):
        self.sample_size = sample_size
        self.random_seed = random_seed
        self.min_shared_verses = min_shared_verses
        np.random.seed(random_seed)
    
    @staticmethod
    def reconstruct_poems_vectorized(df):
        """Vectorized poem reconstruction"""
        valid_mask = df['cluster_id'] != -1
        df_valid = df[valid_mask]
        grouped = df_valid.groupby('idoriginal_poem')['cluster_id'].apply(set)
        return grouped.to_dict()
    
    @staticmethod
    def build_inverted_index_fast(poem_to_clusters):
        """Fast inverted index"""
        cluster_to_poems = defaultdict(set)
        for poem_id, clusters in poem_to_clusters.items():
            for c in clusters:
                cluster_to_poems[c].add(poem_id)
        return cluster_to_poems
    
    @staticmethod
    @njit
    def jaccard_numba(a_arr, b_arr):
        """Ultra-fast Jaccard using Numba"""
        intersection = 0
        a_set = set(a_arr)
        b_set = set(b_arr)
        
        for item in a_set:
            if item in b_set:
                intersection += 1
        
        union = len(a_set) + len(b_set) - intersection
        if union == 0:
            return 0.0
        return intersection / union
    
    @staticmethod
    @njit
    def count_shared_verses(a_arr, b_arr):
        """Count number of shared verse clusters"""
        shared = 0
        a_set = set(a_arr)
        b_set = set(b_arr)
        
        for item in a_set:
            if item in b_set:
                shared += 1
        
        return shared
    
    def stratified_sample_poems(self, df, poem_to_clusters):
        """
        Create a stratified sample of poems based on:
        1. Source dataset distribution (if available)
        2. Poem size (number of verse clusters)
        """
        has_source = 'source_dataset' in df.columns
        
        if has_source:
            poem_to_source = df.groupby('idoriginal_poem')['source_dataset'].first().to_dict()
        
        poem_metadata = []
        for poem_id, clusters in poem_to_clusters.items():
            metadata = {
                'poem_id': poem_id,
                'n_clusters': len(clusters)
            }
            
            if has_source:
                metadata['source'] = poem_to_source.get(poem_id, 'unknown')
            
            poem_metadata.append(metadata)
        
        poem_df = pd.DataFrame(poem_metadata)
        
        # Define size bins
        poem_df['size_bin'] = pd.cut(poem_df['n_clusters'], 
                                      bins=[0, 5, 10, 20, 50, np.inf],
                                      labels=['tiny', 'small', 'medium', 'large', 'huge'])
        
        # Stratified sampling
        sample_indices = []
        
        if has_source:
            print("  Stratifying by source dataset and poem size...")
            for (source, size_bin), group in poem_df.groupby(['source', 'size_bin']):
                n_in_group = len(group)
                proportion = n_in_group / len(poem_df)
                n_sample = max(1, int(self.sample_size * proportion))
                n_sample = min(n_sample, n_in_group)
                
                sampled = group.sample(n=n_sample, random_state=self.random_seed)
                sample_indices.extend(sampled['poem_id'].tolist())
        else:
            print("  Stratifying by poem size only (no source_dataset column)...")
            for size_bin, group in poem_df.groupby('size_bin'):
                n_in_group = len(group)
                proportion = n_in_group / len(poem_df)
                n_sample = max(1, int(self.sample_size * proportion))
                n_sample = min(n_sample, n_in_group)
                
                sampled = group.sample(n=n_sample, random_state=self.random_seed)
                sample_indices.extend(sampled['poem_id'].tolist())
        
        # If we haven't reached sample_size, add random poems
        if len(sample_indices) < self.sample_size:
            remaining = self.sample_size - len(sample_indices)
            available = set(poem_df['poem_id']) - set(sample_indices)
            if available:
                additional = np.random.choice(list(available), 
                                             size=min(remaining, len(available)), 
                                             replace=False)
                sample_indices.extend(additional)
        
        return sample_indices[:self.sample_size]
    
    def find_candidate_pairs_for_sample(self, sample_poems, poem_to_clusters, cluster_to_poems):
        """
        Find candidate pairs within the sample using inverted index
        """
        sample_set = set(sample_poems)
        candidate_pairs = set()
        
        # Use inverted index to find pairs
        for poem_id in tqdm(sample_poems, desc="Building sample pairs"):
            clusters = poem_to_clusters[poem_id]
            
            # Find all poems that share at least one cluster
            potential_matches = set()
            for cluster_id in clusters:
                potential_matches.update(cluster_to_poems[cluster_id])
            
            # Keep only those in sample
            potential_matches = potential_matches & sample_set
            
            # Create pairs
            for other_poem in potential_matches:
                if other_poem > poem_id:  # Avoid duplicates
                    candidate_pairs.add((poem_id, other_poem))
        
        return candidate_pairs
    
    def compute_sample_similarities(self, candidate_pairs, poem_to_array):
        """
        Compute Jaccard similarities for all candidate pairs
        """
        similarities = []
        
        for p1, p2 in tqdm(candidate_pairs, desc="Computing similarities"):
            # Count shared verses first
            shared = self.count_shared_verses(poem_to_array[p1], poem_to_array[p2])
            
            # Only compute Jaccard if minimum shared verses met
            if shared >= self.min_shared_verses:
                sim = self.jaccard_numba(poem_to_array[p1], poem_to_array[p2])
                similarities.append({
                    'poem1': p1,
                    'poem2': p2,
                    'similarity': sim,
                    'shared_verses': shared
                })
        
        return pd.DataFrame(similarities)
    
    class FastUnionFind:
        """Union-Find with path compression for efficient clustering"""
        __slots__ = ['parent', 'rank']
        
        def __init__(self, elements):
            self.parent = {e: e for e in elements}
            self.rank = {e: 0 for e in elements}
        
        def find(self, x):
            if self.parent[x] != x:
                self.parent[x] = self.find(self.parent[x])
            return self.parent[x]
        
        def union(self, x, y):
            px, py = self.find(x), self.find(y)
            if px == py:
                return False
            if self.rank[px] < self.rank[py]:
                px, py = py, px
            self.parent[py] = px
            if self.rank[px] == self.rank[py]:
                self.rank[px] += 1
            return True
        
        def get_clusters(self):
            clusters = defaultdict(set)
            for elem in self.parent.keys():
                clusters[self.find(elem)].add(elem)
            return dict(clusters)
    
    def cluster_at_threshold(self, similarities_df, threshold, sample_poems, poem_to_array):
        """
        Perform clustering at a specific threshold using Union-Find
        """
        # Filter similarities by threshold
        valid_pairs = similarities_df[similarities_df['similarity'] >= threshold]
        
        # Cluster using Union-Find
        uf = self.FastUnionFind(sample_poems)
        
        for _, row in valid_pairs.iterrows():
            uf.union(row['poem1'], row['poem2'])
        
        # Get cluster assignments
        poem_clusters = uf.get_clusters()
        cluster_assignments = {}
        for cluster_id, poems in poem_clusters.items():
            for poem in poems:
                cluster_assignments[poem] = cluster_id
        
        return cluster_assignments
    
    def compute_cluster_cohesion(self, poem_to_array, cluster_assignments):
        """
        Compute average within-cluster similarity (cohesion).
        Higher is better - indicates tight clusters.
        """
        poem_ids = list(poem_to_array.keys())
        cohesions = []
        
        # Group poems by cluster
        clusters = defaultdict(list)
        for poem_id in poem_ids:
            cluster_id = cluster_assignments.get(poem_id)
            if cluster_id is not None:
                clusters[cluster_id].append(poem_id)
        
        # Compute cohesion for each cluster
        for cluster_id, cluster_poems in clusters.items():
            if len(cluster_poems) < 2:
                continue
            
            # Sample pairs if cluster is too large
            if len(cluster_poems) > 30:
                sampled = np.random.choice(cluster_poems, 30, replace=False)
            else:
                sampled = cluster_poems
            
            # Compute average pairwise similarity
            sims = []
            for i in range(len(sampled)):
                for j in range(i+1, len(sampled)):
                    sim = self.jaccard_numba(poem_to_array[sampled[i]], poem_to_array[sampled[j]])
                    sims.append(sim)
            
            if sims:
                cohesions.append(np.mean(sims))
        
        return np.mean(cohesions) if cohesions else 0.0
    
    def compute_cluster_separation(self, poem_to_array, cluster_assignments, n_samples=300):
        """
        Compute average between-cluster dissimilarity (separation).
        Higher is better - indicates well-separated clusters.
        """
        poem_ids = list(poem_to_array.keys())
        unique_clusters = set(cluster_assignments.values())
        
        if len(unique_clusters) < 2:
            return 1.0
        
        separations = []
        cluster_to_poems = defaultdict(list)
        for poem_id, cluster_id in cluster_assignments.items():
            cluster_to_poems[cluster_id].append(poem_id)
        
        # Sample pairs from different clusters
        unique_clusters = list(unique_clusters)
        for _ in range(n_samples):
            c1, c2 = np.random.choice(unique_clusters, 2, replace=False)
            
            p1 = np.random.choice(cluster_to_poems[c1])
            p2 = np.random.choice(cluster_to_poems[c2])
            
            sim = self.jaccard_numba(poem_to_array[p1], poem_to_array[p2])
            separations.append(1 - sim)
        
        return np.mean(separations) if separations else 0.0
    
    def compute_silhouette_approximation(self, poem_to_array, cluster_assignments, n_samples=500):
        """
        Approximate silhouette score using sampling.
        Ranges from -1 to 1, higher is better.
        """
        poem_ids = list(poem_to_array.keys())
        unique_clusters = set(cluster_assignments.values())
        
        if len(unique_clusters) < 2:
            return 0.0
        
        # Sample poems
        if len(poem_ids) > n_samples:
            sampled_poems = np.random.choice(poem_ids, n_samples, replace=False)
        else:
            sampled_poems = poem_ids
        
        silhouettes = []
        cluster_to_poems = defaultdict(list)
        for poem_id, cluster_id in cluster_assignments.items():
            cluster_to_poems[cluster_id].append(poem_id)
        
        for poem_id in sampled_poems:
            cluster_id = cluster_assignments[poem_id]
            same_cluster = [p for p in cluster_to_poems[cluster_id] if p != poem_id]
            
            if len(same_cluster) == 0:
                continue
            
            # Sample if too many
            if len(same_cluster) > 15:
                same_cluster = np.random.choice(same_cluster, 15, replace=False)
            
            # Average distance to same cluster
            a = np.mean([1 - self.jaccard_numba(poem_to_array[poem_id], poem_to_array[p]) 
                        for p in same_cluster])
            
            # Average distance to nearest other cluster
            other_clusters = [c for c in unique_clusters if c != cluster_id]
            if len(other_clusters) == 0:
                continue
            
            min_b = float('inf')
            for other_cluster in other_clusters:
                other_poems = cluster_to_poems[other_cluster]
                
                if len(other_poems) > 15:
                    other_poems = np.random.choice(other_poems, 15, replace=False)
                
                b = np.mean([1 - self.jaccard_numba(poem_to_array[poem_id], poem_to_array[p]) 
                            for p in other_poems])
                min_b = min(min_b, b)
            
            # Silhouette coefficient
            s = (min_b - a) / max(a, min_b) if max(a, min_b) > 0 else 0
            silhouettes.append(s)
        
        return np.mean(silhouettes) if silhouettes else 0.0
    
    def evaluate_threshold(self, threshold, similarities_df, sample_poems, poem_to_array):
        """
        Evaluate a single threshold using quality metrics
        """
        # Cluster at this threshold
        cluster_assignments = self.cluster_at_threshold(
            similarities_df, threshold, sample_poems, poem_to_array
        )
        
        # Compute cluster statistics
        clusters = defaultdict(list)
        for poem_id, cluster_id in cluster_assignments.items():
            clusters[cluster_id].append(poem_id)
        
        n_clusters = len(clusters)
        cluster_sizes = [len(poems) for poems in clusters.values()]
        n_singletons = sum(1 for size in cluster_sizes if size == 1)
        avg_size = np.mean(cluster_sizes) if cluster_sizes else 0
        max_size = max(cluster_sizes) if cluster_sizes else 0
        
        # Compute quality metrics
        cohesion = self.compute_cluster_cohesion(poem_to_array, cluster_assignments)
        separation = self.compute_cluster_separation(poem_to_array, cluster_assignments)
        silhouette = self.compute_silhouette_approximation(poem_to_array, cluster_assignments)
        
        # Count pairs above threshold
        n_pairs_above = len(similarities_df[similarities_df['similarity'] >= threshold])
        pct_pairs_above = (n_pairs_above / len(similarities_df) * 100) if len(similarities_df) > 0 else 0
        
        return {
            'threshold': threshold,
            'n_clusters': n_clusters,
            'n_singletons': n_singletons,
            'avg_cluster_size': avg_size,
            'max_cluster_size': max_size,
            'cohesion': cohesion,
            'separation': separation,
            'silhouette': silhouette,
            'n_pairs_above': n_pairs_above,
            'pct_pairs_above': pct_pairs_above
        }
    
    def grid_search_thresholds(self, similarities_df, sample_poems, poem_to_array):
        """
        Test multiple thresholds and select best based on quality metrics
        """
        print("\n" + "="*70)
        print("GRID SEARCH: TESTING MULTIPLE THRESHOLDS")
        print("="*70)
        
        # Generate threshold range
        thresholds = np.linspace(THRESHOLD_RANGE[0], THRESHOLD_RANGE[1], THRESHOLD_RANGE[2])
        print(f"Testing {len(thresholds)} thresholds from {thresholds[0]:.2f} to {thresholds[-1]:.2f}")
        
        results = []
        for threshold in tqdm(thresholds, desc="Evaluating thresholds"):
            result = self.evaluate_threshold(threshold, similarities_df, sample_poems, poem_to_array)
            results.append(result)
        
        results_df = pd.DataFrame(results)
        
        # Compute composite quality score
        def normalize(series):
            min_val = series.min()
            max_val = series.max()
            if max_val - min_val < 1e-10:
                return pd.Series(0.5, index=series.index)
            return (series - min_val) / (max_val - min_val)
        
        silhouette_score = normalize(results_df['silhouette'])
        cohesion_score = normalize(results_df['cohesion'])
        separation_score = normalize(results_df['separation'])
        
        # Penalize too many singletons
        singleton_ratio = results_df['n_singletons'] / len(sample_poems)
        balance_score = np.clip(1 - singleton_ratio, 0, 1)
        
        # Composite quality score (same weights as semantic script)
        results_df['quality_score'] = (
            silhouette_score * 0.40 +
            cohesion_score * 0.30 +
            separation_score * 0.20 +
            balance_score * 0.10
        )
        
        # Sort by quality
        results_df = results_df.sort_values('quality_score', ascending=False)
        
        # Save results
        results_df.to_csv('full_orthographic_results/poem_threshold_grid_search.csv', index=False)
        print(f"\n✓ Grid search results saved to: poem_threshold_grid_search.csv")
        
        return results_df
    
    def plot_grid_search_analysis(self, results_df, selected_threshold):
        """
        Create comprehensive visualization of grid search results
        """
        sns.set_palette("colorblind")
        fig = plt.figure(figsize=(18, 12))
        gs = fig.add_gridspec(3, 3, hspace=0.35, wspace=0.3)
        
        thresholds = results_df['threshold'].values
        
        # 1. Quality score vs threshold
        ax1 = fig.add_subplot(gs[0, :])
        ax1.plot(thresholds, results_df['quality_score'], 'o-', 
                linewidth=2, markersize=8, color='#0173B2', label='Quality Score')
        ax1.axvline(selected_threshold, color='#CC0000', linestyle='--', linewidth=2,
                   label=f'Selected: {selected_threshold:.3f}')
        ax1.set_xlabel('Threshold', fontweight='bold', fontsize=12)
        ax1.set_ylabel('Quality Score', fontweight='bold', fontsize=12)
        ax1.set_title('Composite Quality Score vs Threshold', fontweight='bold', fontsize=14)
        ax1.legend(fontsize=11)
        ax1.grid(alpha=0.3)
        
        # Mark the best point
        best_idx = results_df['quality_score'].idxmax()
        ax1.scatter(results_df.loc[best_idx, 'threshold'], 
                   results_df.loc[best_idx, 'quality_score'],
                   color='red', s=200, marker='*', edgecolors='black', linewidth=2, zorder=10)
        
        # 2. Individual metrics
        ax2 = fig.add_subplot(gs[1, 0])
        ax2.plot(thresholds, results_df['silhouette'], 'o-', linewidth=2, label='Silhouette', color='#029E73')
        ax2.axvline(selected_threshold, color='#CC0000', linestyle='--', linewidth=1, alpha=0.5)
        ax2.set_xlabel('Threshold', fontweight='bold')
        ax2.set_ylabel('Silhouette Score', fontweight='bold')
        ax2.set_title('Silhouette Score', fontweight='bold')
        ax2.legend()
        ax2.grid(alpha=0.3)
        
        ax3 = fig.add_subplot(gs[1, 1])
        ax3.plot(thresholds, results_df['cohesion'], 'o-', linewidth=2, label='Cohesion', color='#DE8F05')
        ax3.axvline(selected_threshold, color='#CC0000', linestyle='--', linewidth=1, alpha=0.5)
        ax3.set_xlabel('Threshold', fontweight='bold')
        ax3.set_ylabel('Cohesion', fontweight='bold')
        ax3.set_title('Within-Cluster Cohesion', fontweight='bold')
        ax3.legend()
        ax3.grid(alpha=0.3)
        
        ax4 = fig.add_subplot(gs[1, 2])
        ax4.plot(thresholds, results_df['separation'], 'o-', linewidth=2, label='Separation', color='#CC78BC')
        ax4.axvline(selected_threshold, color='#CC0000', linestyle='--', linewidth=1, alpha=0.5)
        ax4.set_xlabel('Threshold', fontweight='bold')
        ax4.set_ylabel('Separation', fontweight='bold')
        ax4.set_title('Between-Cluster Separation', fontweight='bold')
        ax4.legend()
        ax4.grid(alpha=0.3)
        
        # 3. Cluster statistics
        ax5 = fig.add_subplot(gs[2, 0])
        ax5.plot(thresholds, results_df['n_clusters'], 'o-', linewidth=2, color='#0173B2')
        ax5.axvline(selected_threshold, color='#CC0000', linestyle='--', linewidth=1, alpha=0.5)
        ax5.set_xlabel('Threshold', fontweight='bold')
        ax5.set_ylabel('Number of Clusters', fontweight='bold')
        ax5.set_title('Cluster Count', fontweight='bold')
        ax5.grid(alpha=0.3)
        
        ax6 = fig.add_subplot(gs[2, 1])
        ax6.plot(thresholds, results_df['avg_cluster_size'], 'o-', linewidth=2, color='#029E73')
        ax6.axvline(selected_threshold, color='#CC0000', linestyle='--', linewidth=1, alpha=0.5)
        ax6.set_xlabel('Threshold', fontweight='bold')
        ax6.set_ylabel('Average Cluster Size', fontweight='bold')
        ax6.set_title('Average Cluster Size', fontweight='bold')
        ax6.grid(alpha=0.3)
        
        # 4. Multi-metric comparison
        ax7 = fig.add_subplot(gs[2, 2])
        
        # Normalize all metrics for comparison
        def norm(series):
            return (series - series.min()) / (series.max() - series.min() + 1e-10)
        
        ax7.plot(thresholds, norm(results_df['silhouette']), 'o-', label='Silhouette', linewidth=2, alpha=0.7)
        ax7.plot(thresholds, norm(results_df['cohesion']), 's-', label='Cohesion', linewidth=2, alpha=0.7)
        ax7.plot(thresholds, norm(results_df['separation']), '^-', label='Separation', linewidth=2, alpha=0.7)
        ax7.plot(thresholds, results_df['quality_score'], 'd-', label='Quality', linewidth=2.5, color='black')
        ax7.axvline(selected_threshold, color='#CC0000', linestyle='--', linewidth=2)
        ax7.set_xlabel('Threshold', fontweight='bold')
        ax7.set_ylabel('Normalized Score', fontweight='bold')
        ax7.set_title('All Metrics (Normalized)', fontweight='bold')
        ax7.legend()
        ax7.grid(alpha=0.3)
        
        # Overall title
        fig.suptitle(f'Orthographic Poem Threshold Grid Search (min_shared={self.min_shared_verses})', 
                    fontsize=16, fontweight='bold', y=0.995)
        
        plt.savefig('full_orthographic_results/poem_threshold_grid_search_analysis.png', 
                   dpi=300, bbox_inches='tight')
        print(f"✓ Grid search visualization saved")
        plt.close()
    
    def run_threshold_analysis(self, df):
        """
        Main method to run complete threshold analysis with quality metrics
        """
        print("="*70)
        print("ENHANCED POEM-LEVEL THRESHOLD SELECTION")
        print("="*70)
        print(f"Minimum shared verses: {self.min_shared_verses}")
        
        # Step 1: Reconstruct poems
        print("\nStep 1: Reconstructing poems...")
        poem_to_clusters = self.reconstruct_poems_vectorized(df)
        print(f"  Found {len(poem_to_clusters):,} poems")
        
        # Step 2: Build inverted index
        print("\nStep 2: Building inverted index...")
        cluster_to_poems = self.build_inverted_index_fast(poem_to_clusters)
        print(f"  Found {len(cluster_to_poems):,} verse clusters")
        
        # Step 3: Sample poems
        print(f"\nStep 3: Sampling {self.sample_size:,} poems...")
        sample_poems = self.stratified_sample_poems(df, poem_to_clusters)
        print(f"  Sampled {len(sample_poems):,} poems")
        
        # Step 4: Find candidate pairs
        print("\nStep 4: Finding candidate pairs in sample...")
        candidate_pairs = self.find_candidate_pairs_for_sample(
            sample_poems, poem_to_clusters, cluster_to_poems
        )
        print(f"  Found {len(candidate_pairs):,} candidate pairs")
        
        # Step 5: Convert to arrays for fast computation
        print("\nStep 5: Converting to arrays...")
        poem_to_array = {
            p: np.array(sorted(poem_to_clusters[p]), dtype=np.int32)
            for p in sample_poems
        }
        
        # Step 6: Compute similarities
        print(f"\nStep 6: Computing similarities (filtering pairs with <{self.min_shared_verses} shared verses)...")
        similarities_df = self.compute_sample_similarities(candidate_pairs, poem_to_array)
        similarities_df.to_csv('full_orthographic_results/poem_similarities_sample.csv', index=False)
        print(f"  Computed {len(similarities_df):,} similarities (after filtering)")
        
        # Step 7: Grid search over thresholds
        print("\nStep 7: Grid search over thresholds...")
        grid_results = self.grid_search_thresholds(similarities_df, sample_poems, poem_to_array)
        
        # Step 8: Select best threshold
        print("\n" + "="*70)
        print("THRESHOLD SELECTION BASED ON QUALITY METRICS")
        print("="*70)
        
        best_result = grid_results.iloc[0]
        threshold = float(best_result['threshold'])
        
        print(f"Selected Threshold:   {threshold:.4f}")
        print(f"Quality Score:        {best_result['quality_score']:.4f}")
        print(f"Silhouette:           {best_result['silhouette']:.4f}")
        print(f"Cohesion:             {best_result['cohesion']:.4f}")
        print(f"Separation:           {best_result['separation']:.4f}")
        print(f"Clusters:             {int(best_result['n_clusters']):,}")
        print(f"Singletons:           {int(best_result['n_singletons']):,}")
        print(f"Avg Cluster Size:     {best_result['avg_cluster_size']:.2f}")
        print(f"Pairs Above Threshold: {int(best_result['n_pairs_above']):,} ({best_result['pct_pairs_above']:.2f}%)")
        
        # Step 9: Create visualizations
        print("\nStep 9: Creating visualizations...")
        self.plot_grid_search_analysis(grid_results, threshold)
        
        # Save summary
        summary = {
            'selected_threshold': threshold,
            'quality_score': best_result['quality_score'],
            'silhouette': best_result['silhouette'],
            'cohesion': best_result['cohesion'],
            'separation': best_result['separation'],
            'n_clusters': int(best_result['n_clusters']),
            'n_singletons': int(best_result['n_singletons']),
            'avg_cluster_size': best_result['avg_cluster_size'],
            'min_shared_verses': self.min_shared_verses,
            'n_pairs_above': int(best_result['n_pairs_above']),
            'pct_pairs_above': best_result['pct_pairs_above'],
            'sample_size': len(sample_poems),
            'total_poems': len(poem_to_clusters)
        }
        
        pd.DataFrame([summary]).to_csv('full_orthographic_results/poem_enhanced_threshold_summary.csv', index=False)
        print(f"Summary saved to: poem_enhanced_threshold_summary.csv")
        
        print("\n" + "="*70)
        print("ANALYSIS COMPLETE")
        print("="*70)
        
        return threshold, grid_results, similarities_df


# =============================================================================
# MAIN EXECUTION
# =============================================================================
print(f"Loading data from: {INPUT_CSV}")

try:
    df = pd.read_csv(INPUT_CSV)
except FileNotFoundError:
    print(f"Error: File not found: {INPUT_CSV}")
    print(f"\nPlease update the INPUT_CSV variable at the top of this script.")
    raise

# Check required columns
required_cols = ['cluster_id', 'idoriginal_poem']
missing_cols = [col for col in required_cols if col not in df.columns]

if missing_cols:
    print(f"\nError: Missing required columns: {missing_cols}")
    print(f"Available columns: {list(df.columns)}")
    raise ValueError(f"Missing columns: {missing_cols}")

# Ensure correct data types
df['cluster_id'] = pd.to_numeric(df['cluster_id'], errors='coerce').fillna(-1).astype(int)
df['idoriginal_poem'] = df['idoriginal_poem'].astype(str)

print(f"✓ Loaded {len(df):,} verses")
print(f"✓ Unique poems: {df['idoriginal_poem'].nunique():,}")
print(f"✓ Unique verse clusters: {df[df['cluster_id'] != -1]['cluster_id'].nunique():,}")

if 'source_dataset' in df.columns:
    print(f"✓ Source datasets: {df['source_dataset'].unique()}")
else:
    print("ℹ️  No source_dataset column - sampling by poem size only")

# Initialize and run
print("\n" + "="*70)
print("STARTING ENHANCED ANALYSIS")
print("="*70)

selector = EnhancedPoemThresholdSelector(
    sample_size=SAMPLE_SIZE, 
    random_seed=RANDOM_SEED, 
    min_shared_verses=MIN_SHARED_VERSES
)

start_time = time.time()
threshold, grid_results, similarities_df = selector.run_threshold_analysis(df)
elapsed = time.time() - start_time

print(f"\n{'='*70}")
print(f"TOTAL TIME: {elapsed/60:.2f} minutes")
print(f"RECOMMENDED THRESHOLD: {threshold:.4f}")
print(f"{'='*70}")

# Show top 5 configurations
print("\n" + "="*70)
print("TOP 5 CONFIGURATIONS")
print("="*70)
for i, row in grid_results.head(5).iterrows():
    print(f"\n{i+1}. Threshold: {row['threshold']:.3f}")
    print(f"   Quality: {row['quality_score']:.3f}, Silhouette: {row['silhouette']:.3f}")
    print(f"   Clusters: {int(row['n_clusters']):,}, Avg Size: {row['avg_cluster_size']:.2f}")


df = pd.read_csv('full_orthographic_results/poem_threshold_grid_search.csv')

df = df.sort_values('threshold')

plt.figure(figsize=(10, 6))
sns.lineplot(
    data=df,
    x='threshold',
    y='quality_score',
    marker='o',
    linewidth=2,
    markersize=8
)

plt.title(f'Quality Score vs Threshold)',
          fontsize=16, fontweight='bold')
plt.xlabel('Cosine Similarity Threshold', fontsize=14, fontweight='bold')
plt.ylabel('Quality Score', fontsize=14, fontweight='bold')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('full_orthographic_results/poem_quality_vs_threshold.png', dpi=300, bbox_inches='tight')


Loading data from: full_orthographic_results/clustered_optimized.csv


/tmp/ipykernel_3163273/552467930.py:1183: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(INPUT_CSV)


✓ Loaded 1,537,740 verses
✓ Unique poems: 164,665
✓ Unique verse clusters: 1,261,001
✓ Source datasets: ['rhoby' 'dbbe' 'phi' 'papyri']

STARTING ENHANCED ANALYSIS
ENHANCED POEM-LEVEL THRESHOLD SELECTION
Minimum shared verses: 2

Step 1: Reconstructing poems...
  Found 164,665 poems

Step 2: Building inverted index...
  Found 1,261,001 verse clusters

Step 3: Sampling 15,000 poems...
  Stratifying by source dataset and poem size...
  Sampled 15,000 poems

Step 4: Finding candidate pairs in sample...


Building sample pairs: 100%|██████████| 15000/15000 [00:01<00:00, 10430.69it/s]


  Found 1,157,993 candidate pairs

Step 5: Converting to arrays...

Step 6: Computing similarities (filtering pairs with <2 shared verses)...


Computing similarities: 100%|██████████| 1157993/1157993 [00:06<00:00, 189700.35it/s]


  Computed 2,329 similarities (after filtering)

Step 7: Grid search over thresholds...

GRID SEARCH: TESTING MULTIPLE THRESHOLDS
Testing 13 thresholds from 0.30 to 0.90


Evaluating thresholds: 100%|██████████| 13/13 [00:11<00:00,  1.17it/s]



✓ Grid search results saved to: poem_threshold_grid_search.csv

THRESHOLD SELECTION BASED ON QUALITY METRICS
Selected Threshold:   0.8500
Quality Score:        0.8604
Silhouette:           1.0000
Cohesion:             1.0000
Separation:           0.9996
Clusters:             14,991
Singletons:           14,984
Avg Cluster Size:     1.00
Pairs Above Threshold: 11 (0.47%)

Step 9: Creating visualizations...
✓ Grid search visualization saved
Summary saved to: poem_enhanced_threshold_summary.csv

ANALYSIS COMPLETE

TOTAL TIME: 0.42 minutes
RECOMMENDED THRESHOLD: 0.8500

TOP 5 CONFIGURATIONS

12. Threshold: 0.850
   Quality: 0.860, Silhouette: 1.000
   Clusters: 14,991, Avg Size: 1.00

4. Threshold: 0.450
   Quality: 0.481, Silhouette: 0.514
   Clusters: 14,952, Avg Size: 1.00

11. Threshold: 0.800
   Quality: 0.472, Silhouette: 0.000
   Clusters: 14,990, Avg Size: 1.00

6. Threshold: 0.550
   Quality: 0.395, Silhouette: 0.278
   Clusters: 14,969, Avg Size: 1.00

3. Threshold: 0.400
   Qua

In [ ]:
## Applying the threshold
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from numba import njit
import gc
import time
import os


INPUT_CSV = "full_orthographic_results/clustered_optimized.csv"
THRESHOLD = 0.3
MIN_SHARED_VERSES = 2
OUTPUT_DIR = "full_poem_clustering_results"


class FastPoemClusterer:
    def __init__(self, threshold: float = 0.6, min_shared_verses: int = 2):
        self.threshold = threshold
        self.min_shared_verses = min_shared_verses
        self.poem_clusters = {}
        self.similarities = []
    
    @staticmethod
    def reconstruct_poems_vectorized(df):
        valid_mask = df['cluster_id'] != -1
        df_valid = df[valid_mask]
        grouped = df_valid.groupby('idoriginal_poem')['cluster_id'].apply(set)
        return grouped.to_dict()
    
    @staticmethod
    def build_inverted_index_fast(poem_to_clusters):
        cluster_to_poems = defaultdict(set)
        for poem_id, clusters in poem_to_clusters.items():
            for c in clusters:
                cluster_to_poems[c].add(poem_id)
        return cluster_to_poems
    
    @staticmethod
    @njit
    def jaccard_numba(a_arr, b_arr):
        intersection = 0
        a_set = set(a_arr)
        b_set = set(b_arr)
        
        for item in a_set:
            if item in b_set:
                intersection += 1
        
        union = len(a_set) + len(b_set) - intersection
        if union == 0:
            return 0.0
        return intersection / union
    
    @staticmethod
    @njit
    def count_shared_verses(a_arr, b_arr):
        shared = 0
        a_set = set(a_arr)
        b_set = set(b_arr)
        
        for item in a_set:
            if item in b_set:
                shared += 1
        
        return shared
    
    def find_candidate_pairs_optimized(self, poem_to_clusters, cluster_to_poems):
        candidate_pairs = set()
        
        multi_poem_clusters = {
            c: poems for c, poems in cluster_to_poems.items() 
            if len(poems) > 1
        }
        
        print(f"Processing {len(multi_poem_clusters):,} clusters (filtered from {len(cluster_to_poems):,})")
        
        small_threshold = 1000
        
        for cluster_id, poems in tqdm(multi_poem_clusters.items(), desc="Building candidates"):
            poems_list = list(poems)
            n_poems = len(poems_list)
            
            if n_poems <= small_threshold:
                for i in range(n_poems):
                    for j in range(i + 1, n_poems):
                        candidate_pairs.add(tuple(sorted((poems_list[i], poems_list[j]))))
            else:
                print(f"  Warning: Large cluster {cluster_id} with {n_poems} poems - sampling pairs")
                indices = np.random.choice(n_poems, size=min(2000, n_poems), replace=False)
                for i in range(len(indices)):
                    for j in range(i + 1, len(indices)):
                        candidate_pairs.add(tuple(sorted((poems_list[indices[i]], poems_list[indices[j]]))))
        
        return candidate_pairs
    
    def cluster_poems_fast(self, df):
        print("Reconstructing poems...")
        poem_to_clusters = self.reconstruct_poems_vectorized(df)
        print(f"Found {len(poem_to_clusters):,} poems")
        
        print("Building inverted index...")
        cluster_to_poems = self.build_inverted_index_fast(poem_to_clusters)
        print(f"Found {len(cluster_to_poems):,} clusters")
        
        print("Finding candidate pairs...")
        candidate_pairs = self.find_candidate_pairs_optimized(poem_to_clusters, cluster_to_poems)
        print(f"Found {len(candidate_pairs):,} candidate pairs")
        
        print("Converting to arrays...")
        poem_to_array = {
            p: np.array(sorted(clusters), dtype=np.int32) 
            for p, clusters in poem_to_clusters.items()
        }
        
        poem_ids = list(poem_to_clusters.keys())
        parent = {p: p for p in poem_ids}
        rank = {p: 0 for p in poem_ids}
        
        def find(x):
            if parent[x] != x:
                parent[x] = find(parent[x])
            return parent[x]
        
        def union(x, y):
            px, py = find(x), find(y)
            if px == py:
                return False
            
            if rank[px] < rank[py]:
                px, py = py, px
            parent[py] = px
            if rank[px] == rank[py]:
                rank[px] += 1
            return True
        
        print(f"Computing similarities and clustering (min shared verses: {self.min_shared_verses})...")
        similarities = []
        merges = 0
        filtered_by_shared = 0
        
        batch_size = 100000
        pairs_list = list(candidate_pairs)
        
        for batch_start in tqdm(range(0, len(pairs_list), batch_size), desc="Processing batches"):
            batch_end = min(batch_start + batch_size, len(pairs_list))
            batch_pairs = pairs_list[batch_start:batch_end]
            
            for p1, p2 in batch_pairs:
                if find(p1) == find(p2):
                    continue
                
                shared = self.count_shared_verses(poem_to_array[p1], poem_to_array[p2])
                
                if shared < self.min_shared_verses:
                    filtered_by_shared += 1
                    continue
                
                sim = self.jaccard_numba(poem_to_array[p1], poem_to_array[p2])
                
                if sim >= self.threshold:
                    if union(p1, p2):
                        merges += 1
                    similarities.append((p1, p2, sim, shared))
            
            if batch_start % (batch_size * 5) == 0:
                gc.collect()
        
        print(f"Merged {merges:,} poem pairs")
        print(f"Filtered {filtered_by_shared:,} pairs with <{self.min_shared_verses} shared verses")
        
        self.poem_clusters = {p: find(p) for p in poem_ids}
        self.similarities = similarities
        
        return self.poem_clusters, self.similarities
    
    def cluster_sizes(self):
        cluster_count = defaultdict(int)
        for cluster in self.poem_clusters.values():
            cluster_count[cluster] += 1
        sizes = np.array(list(cluster_count.values()))
        return sizes


class UltraFastPoemClusterer(FastPoemClusterer):
    
    def find_candidate_pairs_with_size_filter(self, poem_to_clusters, cluster_to_poems):
        max_ratio = (1 + self.threshold) / self.threshold
        candidate_pairs = set()
        poem_sizes = {p: len(clusters) for p, clusters in poem_to_clusters.items()}
        
        multi_poem_clusters = {
            c: poems for c, poems in cluster_to_poems.items() 
            if len(poems) > 1
        }
        
        print(f"Processing {len(multi_poem_clusters):,} clusters")
        print(f"Max size ratio filter: {max_ratio:.2f}x")
        
        filtered_by_size = 0
        
        for cluster_id, poems in tqdm(multi_poem_clusters.items(), desc="Building candidates"):
            poems_list = list(poems)
            n_poems = len(poems_list)
            
            if n_poems > 1000:
                print(f"  Warning: Large cluster {cluster_id} with {n_poems} poems")
                poems_list = list(np.random.choice(poems_list, size=min(1000, n_poems), replace=False))
                n_poems = len(poems_list)
            
            for i in range(n_poems):
                p1 = poems_list[i]
                size1 = poem_sizes[p1]
                
                for j in range(i + 1, n_poems):
                    p2 = poems_list[j]
                    size2 = poem_sizes[p2]
                    
                    ratio = max(size1, size2) / max(min(size1, size2), 1)
                    if ratio > max_ratio:
                        filtered_by_size += 1
                        continue
                    
                    candidate_pairs.add(tuple(sorted((p1, p2))))
        
        print(f"Filtered {filtered_by_size:,} pairs by size")
        
        return candidate_pairs
    
    def cluster_poems_ultrafast(self, df):
        print("Reconstructing poems...")
        poem_to_clusters = self.reconstruct_poems_vectorized(df)
        print(f"Found {len(poem_to_clusters):,} poems")
        
        print("Building inverted index...")
        cluster_to_poems = self.build_inverted_index_fast(poem_to_clusters)
        print(f"Found {len(cluster_to_poems):,} clusters")
        
        print("Finding candidate pairs with size filter...")
        candidate_pairs = self.find_candidate_pairs_with_size_filter(
            poem_to_clusters, cluster_to_poems
        )
        print(f"Found {len(candidate_pairs):,} candidate pairs")
        
        return self._cluster_with_pairs(poem_to_clusters, candidate_pairs)
    
    def _cluster_with_pairs(self, poem_to_clusters, candidate_pairs):
        poem_to_array = {
            p: np.array(sorted(clusters), dtype=np.int32) 
            for p, clusters in poem_to_clusters.items()
        }
        
        poem_ids = list(poem_to_clusters.keys())
        parent = {p: p for p in poem_ids}
        rank = {p: 0 for p in poem_ids}
        
        def find(x):
            if parent[x] != x:
                parent[x] = find(parent[x])
            return parent[x]
        
        def union(x, y):
            px, py = find(x), find(y)
            if px == py:
                return False
            if rank[px] < rank[py]:
                px, py = py, px
            parent[py] = px
            if rank[px] == rank[py]:
                rank[px] += 1
            return True
        
        print(f"Computing similarities (min shared verses: {self.min_shared_verses})...")
        similarities = []
        merges = 0
        filtered_by_shared = 0
        
        pairs_list = list(candidate_pairs)
        
        for p1, p2 in tqdm(pairs_list, desc="Clustering"):
            if find(p1) == find(p2):
                continue
            
            shared = self.count_shared_verses(poem_to_array[p1], poem_to_array[p2])
            
            if shared < self.min_shared_verses:
                filtered_by_shared += 1
                continue
            
            sim = self.jaccard_numba(poem_to_array[p1], poem_to_array[p2])
            
            if sim >= self.threshold:
                if union(p1, p2):
                    merges += 1
                similarities.append((p1, p2, sim, shared))
        
        print(f"Merged {merges:,} poem pairs")
        print(f"Filtered {filtered_by_shared:,} pairs with <{self.min_shared_verses} shared verses")
        
        self.poem_clusters = {p: find(p) for p in poem_ids}
        self.similarities = similarities
        
        return self.poem_clusters, self.similarities


if __name__ == "__main__":
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    print("="*70)
    print("FULL POEM CLUSTERING")
    print("="*70)
    print(f"Input: {INPUT_CSV}")
    print(f"Threshold: {THRESHOLD}")
    print(f"Min shared verses: {MIN_SHARED_VERSES}")
    print(f"Output directory: {OUTPUT_DIR}/")
    print("="*70)
    
    print("\nLoading data...")
    df = pd.read_csv(INPUT_CSV)
    df['cluster_id'] = pd.to_numeric(df['cluster_id'], errors='coerce').fillna(-1).astype(int)
    df['idoriginal_poem'] = df['idoriginal_poem'].astype(str)
    
    print(f"Loaded {len(df):,} verses")
    print(f"Unique poems: {df['idoriginal_poem'].nunique():,}")
    
    n_unique_poems = df['idoriginal_poem'].nunique()
    
    if n_unique_poems > 100000:
        print("\nUsing UltraFastPoemClusterer (size-filtered)")
        clusterer = UltraFastPoemClusterer(threshold=THRESHOLD, min_shared_verses=MIN_SHARED_VERSES)
        start = time.time()
        poem_clusters, similarities = clusterer.cluster_poems_ultrafast(df)
    else:
        print("\nUsing FastPoemClusterer")
        clusterer = FastPoemClusterer(threshold=THRESHOLD, min_shared_verses=MIN_SHARED_VERSES)
        start = time.time()
        poem_clusters, similarities = clusterer.cluster_poems_fast(df)
    
    elapsed = time.time() - start
    
    sizes = clusterer.cluster_sizes()
    
    print(f"\n{'='*70}")
    print("RESULTS")
    print(f"{'='*70}")
    print(f"Time:                  {elapsed/60:.1f} minutes")
    print(f"Poem clusters:         {len(set(poem_clusters.values())):,}")
    print(f"Poems with matches:    {len(similarities):,} pairs")
    print(f"Cluster size mean:     {sizes.mean():.2f}")
    print(f"Cluster size median:   {np.median(sizes):.0f}")
    print(f"Cluster size max:      {sizes.max()}")
    print(f"{'='*70}")
    
    print("\nAdding poem cluster IDs to dataframe...")
    df['poem_cluster_id'] = df['idoriginal_poem'].map(poem_clusters)
    
    print("\nSaving results...")
    
    output_csv = os.path.join(OUTPUT_DIR, "verses_with_poem_clusters.csv")
    df.to_csv(output_csv, index=False)
    print(f"✓ Full results: {output_csv}")
    
    poem_cluster_summary = []
    cluster_to_poems = defaultdict(list)
    
    for poem_id, cluster_id in poem_clusters.items():
        cluster_to_poems[cluster_id].append(poem_id)
    
    for cluster_id, poems in cluster_to_poems.items():
        if len(poems) > 1:
            poem_cluster_summary.append({
                'poem_cluster_id': cluster_id,
                'n_poems': len(poems),
                'poem_ids': ', '.join(poems) if len(poems) <= 10 else f"{', '.join(poems[:10])}... (and {len(poems)-10} more)"
            })
    
    summary_df = pd.DataFrame(poem_cluster_summary).sort_values('n_poems', ascending=False)
    summary_csv = os.path.join(OUTPUT_DIR, "poem_clusters_summary.csv")
    summary_df.to_csv(summary_csv, index=False)
    print(f"✓ Cluster summary: {summary_csv}")
    
    if len(similarities) > 0:
        sim_df = pd.DataFrame(similarities, columns=['poem1', 'poem2', 'similarity', 'shared_verses'])
        sim_df = sim_df.sort_values('similarity', ascending=False)
        sim_csv = os.path.join(OUTPUT_DIR, "poem_similarities.csv")
        sim_df.to_csv(sim_csv, index=False)
        print(f"✓ Similarities: {sim_csv}")
    
    metrics = {
        'threshold': THRESHOLD,
        'min_shared_verses': MIN_SHARED_VERSES,
        'n_poems': n_unique_poems,
        'n_clusters': len(set(poem_clusters.values())),
        'n_multi_poem_clusters': np.sum(sizes > 1),
        'n_singleton_clusters': np.sum(sizes == 1),
        'avg_cluster_size': float(sizes.mean()),
        'median_cluster_size': float(np.median(sizes)),
        'max_cluster_size': int(sizes.max()),
        'n_similarity_pairs': len(similarities),
        'clustering_time_minutes': elapsed / 60
    }
    
    metrics_df = pd.DataFrame([metrics])
    metrics_csv = os.path.join(OUTPUT_DIR, "clustering_metrics.csv")
    metrics_df.to_csv(metrics_csv, index=False)
    print(f"✓ Metrics: {metrics_csv}")
    
    print(f"\n{'='*70}")
    print("TOP 20 POEM CLUSTERS")
    print(f"{'='*70}")
    if len(summary_df) > 0:
        print(summary_df.head(20).to_string(index=False))
    else:
        print("No multi-poem clusters found")
    
    print(f"\n{'='*70}")
    print("✅ CLUSTERING COMPLETE")
    print(f"{'='*70}")
    print(f"All results saved to: {OUTPUT_DIR}/")

In [ ]:
## Checking results
import pandas as pd
from IPython.display import display, Markdown

# === CONFIG ===
OUTPUT_DIR = "full_poem_clustering_results"
N_CLUSTERS_TO_SHOW = 5       # how many poem clusters to display
N_POEMS_PER_CLUSTER = 3      # how many poems to sample per cluster
MAX_LINES_PER_POEM = 20      # limit printed lines per poem (for readability)

# === LOAD RESULTS ===
df = pd.read_csv(f"{OUTPUT_DIR}/verses_with_poem_clusters.csv")
summary_df = pd.read_csv(f"{OUTPUT_DIR}/poem_clusters_summary.csv")

# --- Coerce types safely ---
df['idoriginal_poem'] = df['idoriginal_poem'].astype(str)
if 'poem_cluster_id' in df.columns:
    df['poem_cluster_id'] = df['poem_cluster_id'].astype(str)
if 'poem_cluster_id' in summary_df.columns:
    summary_df['poem_cluster_id'] = summary_df['poem_cluster_id'].astype(str)
if 'source_dataset' in df.columns:
    df['source_dataset'] = df['source_dataset'].astype(str)
if 'order' in df.columns:
    df['order'] = pd.to_numeric(df['order'], errors='coerce').fillna(0)

print(f"Loaded {len(df):,} verses from {df['idoriginal_poem'].nunique():,} poems.")
print(f"Found {len(summary_df):,} multi-poem clusters.\n")

# === RECONSTRUCT FULL POEMS ===
sort_cols = ['idoriginal_poem', 'order'] if 'order' in df.columns else ['idoriginal_poem']
df_sorted = df.sort_values(sort_cols)

poems_df = (
    df_sorted.groupby('idoriginal_poem').agg({
        'verse': lambda x: [v for v in x if isinstance(v, str)],
        'source_dataset': 'first'
    }).reset_index()
)
poems_df['full_poem'] = poems_df['verse'].apply(lambda verses: "\n".join(verses))

# Map poem_cluster_id
poem_to_cluster = (
    df_sorted.drop_duplicates('idoriginal_poem')[['idoriginal_poem', 'poem_cluster_id']]
)
poems_df = poems_df.merge(poem_to_cluster, on='idoriginal_poem', how='left')

print(f"Reconstructed {len(poems_df):,} poems with full text.\n")

# === DISPLAY SAMPLE CLUSTERS ===
sample_clusters = summary_df.head(N_CLUSTERS_TO_SHOW)['poem_cluster_id'].tolist()

for cluster_id in sample_clusters:
    cluster_poems = poems_df[poems_df['poem_cluster_id'] == cluster_id]
    n_poems = len(cluster_poems)
    display(Markdown(f"## 🧩 Cluster `{cluster_id}` — {n_poems} poems"))

    sample_poems = cluster_poems.sample(min(N_POEMS_PER_CLUSTER, n_poems), random_state=42)
    for _, row in sample_poems.iterrows():
        verses = row['full_poem'].split("\n")
        display(Markdown(f"**📜 Poem ID:** `{row['idoriginal_poem']}` | **Dataset:** `{row['source_dataset']}`"))
        
        # Ensure each verse is on a new line in Markdown
        md_verses = "<br>".join(verses[:MAX_LINES_PER_POEM])
        display(Markdown(md_verses))
        
        if len(verses) > MAX_LINES_PER_POEM:
            display(Markdown(f"... _(truncated, total {len(verses)} verses)_"))
    print("-" * 80)
